<a href="https://colab.research.google.com/github/Animeesh1/Daily-run-test/blob/main/Nifty_200_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===========================
# CELL 1 — Install & Imports
# ===========================
!pip -q install yfinance pandas numpy

import os, time, math, datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
import requests
from io import StringIO

pd.options.display.float_format = '{:,.4f}'.format
print("✅ Cell 1 ready")


✅ Cell 1 ready


In [2]:
# ==========================================
# CELL 2 — Config (uses your YahooSymbol)
# ==========================================
# Your sheet (public to 'Anyone with the link: Viewer')
SHEET_ID = "1AEIpCI5prMzJ-NEa1A69c5yPVmTEmgYo3mjnqtzakBo"
GID      = "995974729"
SHEET_CSV_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={GID}"

# Dates & output files
RUN_DATE        = dt.date.today().strftime("%Y-%m-%d")
SANITY_OUT      = f"sanity_prices_{RUN_DATE}.csv"
PRICE_FAIL_OUT  = f"tickers_price_fetch_failed_{RUN_DATE}.csv"
TECHNICALS_OUT  = f"technicals_{RUN_DATE}.csv"
TECH_ERR_OUT    = f"technicals_errors_{RUN_DATE}.csv"
SCORES_OUT      = f"momentum_scores_{RUN_DATE}.csv"
LOG_OUT         = "momentum_log.csv"
TREND_OUT       = "momentum_trend.csv"
ZSCORES_OUT     = f"momentum_zscores_{RUN_DATE}.csv"
SECTOR_RS_OUT   = f"sector_rs_{RUN_DATE}.csv"
TARGETS_OUT     = f"target_weights_{RUN_DATE}.csv"
REBAL_OUT       = f"rebalance_plan_{RUN_DATE}.csv"
PORTF_LOG_OUT   = "portfolio_momentum_log.csv"
BT6M_PICKS_OUT  = f"bt6m_picks_{RUN_DATE}.csv"
BT6M_LOG_OUT    = "bt6m_log.csv"

# Benchmark & windows
BENCHMARK_TICKER = "^NSEI"
WIN_1M, WIN_3M, WIN_6M = 21, 63, 126

# 10-pt momentum score weights
W_TREND, W_RS, W_VOL, W_BRK, W_SEC = 4.0, 2.0, 1.5, 1.5, 1.0

# Composite (fund-style) z-score weights
WZ_12_1, WZ_TREND, WZ_RS, WZ_BRK, WZ_VOL, WZ_SECTOR = 0.50, 0.20, 0.15, 0.10, 0.05, 0.10

# Target-weight constraints
SINGLE_NAME_CAP = 0.10   # ≤10% per stock
SECTOR_CAP      = 0.25   # ≤25% per sector
TURNOVER_CAP    = 0.25   # ≤25% one-way turnover per rebalance
TOP_N           = None   # None = use all; or cap by top-N Z_Total

print("✅ Cell 2 ready | CSV URL:", SHEET_CSV_URL)


✅ Cell 2 ready | CSV URL: https://docs.google.com/spreadsheets/d/1AEIpCI5prMzJ-NEa1A69c5yPVmTEmgYo3mjnqtzakBo/export?format=csv&gid=995974729


In [3]:
# ===========================================
# CELL 3 — Helpers (scoring utilities)
# ===========================================
def pct_change(series: pd.Series, window: int) -> float:
    if series is None or len(series) < window + 1:
        return np.nan
    return float(series.iloc[-1] / series.iloc[-(window+1)] - 1.0)

def within_band(value: float, ref: float, band: float = 0.01) -> bool:
    if ref == 0 or np.isnan(value) or np.isnan(ref): return False
    return abs(value - ref) / abs(ref) <= band

def price_trend_score(close: float, sma20: float, sma50: float, sma200: float) -> float:
    score = 0.0
    for sma in [sma20, sma50, sma200]:
        if np.isnan(sma): continue
        if close > sma: score += 1.0
        elif within_band(close, sma): score += 0.5
    if not any(np.isnan(x) for x in [sma20, sma50, sma200]) and (sma20 > sma50 > sma200):
        score += 1.0
    return max(0.0, min(4.0, score))

def relative_strength_score(stock_close: pd.Series, bench_close: pd.Series) -> float:
    s1, s3 = pct_change(stock_close, WIN_1M), pct_change(stock_close, WIN_3M)
    b1, b3 = pct_change(bench_close, WIN_1M), pct_change(bench_close, WIN_3M)
    score = 0.0
    if not np.isnan(s1) and not np.isnan(b1):
        d = s1 - b1
        score += 1.0 if d >= 0.05 else (0.5 if d >= 0.0 else 0.0)
    if not np.isnan(s3) and not np.isnan(b3):
        d = s3 - b3
        score += 1.0 if d >= 0.05 else (0.5 if d >= 0.0 else 0.0)
    return max(0.0, min(2.0, score))

def volume_momentum_score(vol20: float, vol50: float) -> float:
    if np.isnan(vol20) or np.isnan(vol50) or vol50 == 0: return 0.0
    ratio = vol20 / vol50
    base = 1.0 if ratio > 1.2 else (0.5 if ratio > 1.0 else 0.0)
    bonus = 0.5 if ratio > 1.5 else 0.0
    return max(0.0, min(1.5, base + bonus))

def breakout_structure_score(close: float, high_52w: float, high_3m: float) -> float:
    if np.isnan(high_52w) or high_52w == 0: return 0.0
    score = 0.0
    dist = (high_52w - close) / high_52w
    if dist <= 0.02: score += 1.0
    elif dist <= 0.05: score += 0.5
    if not np.isnan(high_3m) and close > high_3m: score += 0.5
    return max(0.0, min(1.5, score))

def fetch_history(symbol: str, period: str = "400d") -> pd.DataFrame:
    df = yf.Ticker(symbol).history(period=period, auto_adjust=False)
    return df.rename(columns=str.lower)
print("✅ Cell 3 ready")


✅ Cell 3 ready


In [4]:
# =====================================================================
# CELL 4 — Load sheet, standardize columns, sanity prices (no resolver)
# =====================================================================
print("Fetching:", SHEET_CSV_URL)
r = requests.get(SHEET_CSV_URL, timeout=20)
print("HTTP:", r.status_code, r.headers.get("Content-Type"))
r.raise_for_status()

df_raw = pd.read_csv(StringIO(r.text))

# Flexible header matching (case-insensitive)
def pick(colnames, *cands):
    low = {c.lower(): c for c in colnames}
    for cand in cands:
        if cand.lower() in low: return low[cand.lower()]
    return None

col_company   = pick(df_raw.columns, "Company")
col_symbol    = pick(df_raw.columns, "YahooSymbol", "Symbol", "Ticker")
col_quantity  = pick(df_raw.columns, "Quantity", "Qty")
col_sector    = pick(df_raw.columns, "Sector", "Secor")
col_industry  = pick(df_raw.columns, "Industry")

needed = [col_company, col_symbol, col_quantity]
if any(c is None for c in needed):
    raise RuntimeError(f"Missing required columns. Found: {list(df_raw.columns)}; need Company, YahooSymbol, Quantity.")

df_resolved = pd.DataFrame({
    "Company":  df_raw[col_company].astype(str).str.upper().str.strip(),
    "Ticker":   df_raw[col_symbol].astype(str).str.strip(),
    "Quantity": pd.to_numeric(df_raw[col_quantity], errors="coerce"),
    "Sector":   df_raw[col_sector].astype(str).str.strip() if col_sector else "UNKNOWN",
    "Industry": df_raw[col_industry].astype(str).str.strip() if col_industry else "",
}).dropna(subset=["Company"]).reset_index(drop=True)

print("✅ Using Sector/Industry from input sheet only.")
print("Sectors:", df_resolved["Sector"].nunique(), "| Industries:", df_resolved["Industry"].nunique())

# Sanity price pull
def fetch_last_close(sym: str):
    try:
        h = yf.Ticker(sym).history(period="5d")
        if h.empty: return np.nan, "empty_history"
        return float(h["Close"].iloc[-1]), ""
    except Exception as e:
        return np.nan, f"error: {repr(e)}"

sanity_rows, price_fail_rows = [], []
for _, rr in df_resolved.iterrows():
    last, note = fetch_last_close(rr["Ticker"])
    row = {"Company": rr["Company"], "Ticker": rr["Ticker"], "Sector": rr["Sector"],
           "Industry": rr["Industry"], "LatestPrice": last, "Note": note}
    sanity_rows.append(row)
    if pd.isna(last): price_fail_rows.append(row)

sanity_df = pd.DataFrame(sanity_rows).sort_values(["Sector","Company"])
sanity_df.to_csv(SANITY_OUT, index=False)
print(f"💾 Sanity prices saved: {SANITY_OUT} | rows: {len(sanity_df)}")
display(sanity_df.head(20))

if price_fail_rows:
    pd.DataFrame(price_fail_rows).to_csv(PRICE_FAIL_OUT, index=False)
    print(f"⚠️ Price fetch failures saved: {PRICE_FAIL_OUT} (n={len(price_fail_rows)})")

print("✅ Cell 4 done | companies:", df_resolved['Company'].nunique())


Fetching: https://docs.google.com/spreadsheets/d/1AEIpCI5prMzJ-NEa1A69c5yPVmTEmgYo3mjnqtzakBo/export?format=csv&gid=995974729
HTTP: 200 text/csv
✅ Using Sector/Industry from input sheet only.
Sectors: 12 | Industries: 63
💾 Sanity prices saved: sanity_prices_2025-09-28.csv | rows: 200


,Company,Ticker,Sector,Industry,LatestPrice,Note
188,ACC,ACC.NS,Basic Materials,Building Materials,"1,824.4000",
62,AMBUJACEM,AMBUJACEM.NS,Basic Materials,Building Materials,565.2000,
172,APLAPOLLO,APLAPOLLO.NS,Basic Materials,Steel,"1,660.1000",
34,ASIANPAINT,ASIANPAINT.NS,Basic Materials,Specialty Chemicals,"2,342.8000",
44,GRASIM,GRASIM.NS,Basic Materials,Building Materials,"2,746.8000",
50,HINDALCO,HINDALCO.NS,Basic Materials,Aluminum,743.8500,
46,HINDZINC,HINDZINC.NS,Basic Materials,Other Industrial Metals & Mining,449.1000,
94,JINDALSTEL,JINDALSTEL.NS,Basic Materials,Steel,"1,029.5000",
30,JSWSTEEL,JSWSTEEL.NS,Basic Materials,Steel,"1,130.8000",
184,NATIONALUM,NATIONALUM.NS,Basic Materials,Aluminum,200.5800,


✅ Cell 4 done | companies: 200


In [5]:
# ==========================================================================
# CELL 5 — Technicals per YahooSymbol (with long history + suffix fallback)
# ==========================================================================

# Pull more history so 12-1 momentum + 200DMA have enough lookback
HIST_PERIOD_STOCKS = "1500d"   # ~6 years for stocks; use "max" if you prefer
BENCHMARK_PERIOD   = "max"     # full history for the benchmark

def get_history_resilient(sym: str, period: str = HIST_PERIOD_STOCKS):
    """Try the provided Yahoo symbol; if empty history, try common NSE/BSE/base fallbacks."""
    if not sym or str(sym).lower() in ("nan", "none"):
        return None, pd.DataFrame()
    base = sym.split(".")[0]
    suffix = sym.split(".")[1] if "." in sym else ""

    # Try original, then NSE (.NS), then bare, then BSE (.BO)
    candidates = [sym]
    if suffix != "NS": candidates.append(base + ".NS")
    candidates.append(base)
    if suffix != "BO": candidates.append(base + ".BO")

    seen, ordered = set(), []
    for c in candidates:
        if c and c not in seen:
            ordered.append(c); seen.add(c)

    for c in ordered:
        try:
            dfh = yf.Ticker(c).history(period=period, interval="1d", auto_adjust=False)
            if not dfh.empty:
                return c, dfh.rename(columns=str.lower)
        except Exception:
            pass
        time.sleep(0.15)
    return None, pd.DataFrame()

# ---- Benchmark (longer history) ----
try:
    bench_df = fetch_history(BENCHMARK_TICKER, period=BENCHMARK_PERIOD)  # uses Cell 3's fetch_history
    bench_close = bench_df["close"]
except Exception as e:
    bench_close = None
    print("⚠️ Benchmark fetch failed:", e)

# ---- Per-stock technicals ----
tech_rows, err_rows, closes_store = [], [], {}

for _, r in df_resolved.iterrows():
    comp, sym = r["Company"], r["Ticker"]
    rsym, hist = get_history_resilient(sym, period=HIST_PERIOD_STOCKS)
    if hist.empty:
        err_rows.append({"Company": comp, "Ticker": sym, "Error": "no_history"})
        continue

    close = hist["close"]
    vol   = hist["volume"] if "volume" in hist.columns else pd.Series(dtype=float)

    tech_rows.append({
        "Company": comp,
        "Ticker": rsym,
        "Close": float(close.iloc[-1]),
        "SMA20":  float(close.rolling(20).mean().iloc[-1])  if len(close) >= 20  else np.nan,
        "SMA50":  float(close.rolling(50).mean().iloc[-1])  if len(close) >= 50  else np.nan,
        "SMA200": float(close.rolling(200).mean().iloc[-1]) if len(close) >= 200 else np.nan,
        "AvgVol20": float(vol.rolling(20).mean().iloc[-1])  if len(vol)   >= 20  else np.nan,
        "AvgVol50": float(vol.rolling(50).mean().iloc[-1])  if len(vol)   >= 50  else np.nan,
        "Ret1M": pct_change(close, WIN_1M),
        "Ret3M": pct_change(close, WIN_3M),
        "High3M":  float(close.rolling(WIN_3M).max().iloc[-1])   if len(close) >= WIN_3M else float(close.max()),
        "High52W": float(close.rolling(252).max().iloc[-1])      if len(close) >= 252     else float(close.max()),
        "Low52W":  float(close.rolling(252).min().iloc[-1])      if len(close) >= 252     else float(close.min()),
        "Sector": r.get("Sector", "UNKNOWN"),
        "Industry": r.get("Industry", ""),
    })
    closes_store[comp] = close

tech_df = pd.DataFrame(tech_rows).sort_values("Company")
tech_df.to_csv(TECHNICALS_OUT, index=False)
print(f"💾 Technicals saved: {TECHNICALS_OUT} | rows: {len(tech_df)}")
display(tech_df.head(20))

if err_rows:
    tech_err_df = pd.DataFrame(err_rows)
    tech_err_df.to_csv(TECH_ERR_OUT, index=False)
    print(f"⚠️ Technicals errors saved: {TECH_ERR_OUT} (n={len(tech_err_df)})")
else:
    print("✅ No technical fetch errors")

print("✅ Cell 5 done")


💾 Technicals saved: technicals_2025-09-28.csv | rows: 200


,Company,Ticker,Close,SMA20,SMA50,SMA200,AvgVol20,AvgVol50,Ret1M,Ret3M,High3M,High52W,Low52W,Sector,Industry
87,ABB,ABB.NS,"5,173.3999","5,241.6199","5,258.4780","5,720.8182","232,675.2000","310,264.6200",0.0346,-0.1492,"5,949.5000","8,783.0000","4,935.3999",Industrials,Specialty Industrial Machinery
130,ABCAPITAL,ABCAPITAL.NS,278.9000,286.1075,277.8520,219.6171,"4,619,052.0500","5,024,011.6800",0.0052,0.0083,291.9500,291.9500,152.3200,Financial Services,Financial Conglomerates
199,ABFRL,ABFRL.NS,84.9000,87.9100,80.9016,181.3228,"13,756,704.5000","11,707,397.7000",0.0873,0.1320,93.4800,352.3500,72.4700,Consumer Cyclical,Apparel Manufacturing
188,ACC,ACC.NS,"1,824.4000","1,850.1250","1,841.2100","1,924.6027","214,229.5000","323,860.7200",0.0130,-0.0484,"2,001.5000","2,513.4500","1,782.7000",Basic Materials,Building Materials
109,ADANIENSOL,ADANIENSOL.NS,879.2000,828.6625,819.7320,820.8470,"2,519,341.1000","2,136,706.9200",0.1404,-0.0051,937.4000,"1,076.3000",600.7500,Utilities,Utilities - Independent Power Producers
29,ADANIENT,ADANIENT.NS,"2,543.7000","2,415.8700","2,400.1960","2,403.2263","1,662,553.4500","1,147,282.1600",0.1180,-0.0289,"2,676.3999","3,186.1001","2,096.0000",Energy,Thermal Coal
58,ADANIGREEN,ADANIGREEN.NS,"1,047.4000",996.5650,979.5270,973.0243,"5,151,624.8500","3,039,219.0400",0.1371,0.0213,"1,152.1500","2,066.7500",768.5500,Utilities,Utilities - Renewable
26,ADANIPORTS,ADANIPORTS.NS,"1,392.0000","1,386.2550","1,371.9620","1,278.6242","2,385,805.3500","1,982,868.4000",0.0582,-0.0401,"1,456.7000","1,473.8000","1,053.0500",Industrials,Marine Shipping
36,ADANIPOWER,ADANIPOWER.NS,146.2500,133.4255,123.9982,110.9248,"88,561,494.3000","43,149,617.0200",0.2291,0.2484,170.2500,170.2500,87.5300,Utilities,Utilities - Independent Power Producers
142,ALKEM,ALKEM.NS,"5,433.0000","5,417.9250","5,248.7980","5,081.0990","177,298.1000","167,074.2000",0.0106,0.1000,"5,576.0000","6,294.1001","4,563.7002",Healthcare,Drug Manufacturers - Specialty & Generic


✅ No technical fetch errors
✅ Cell 5 done


In [6]:
# ===============================================
# CELL 5.5 — Sector momentum from your portfolio
# ===============================================
assert 'df_resolved' in globals() and 'closes_store' in globals(), "Run Cells 4 and 5 first."

def _safe_ret(comp: str, window: int) -> float:
    s = closes_store.get(comp)
    if s is None or len(s) < window + 1:
        return np.nan
    return float(s.iloc[-1] / s.iloc[-(window+1)] - 1.0)

b1 = pct_change(bench_close, WIN_1M) if bench_close is not None else np.nan
b3 = pct_change(bench_close, WIN_3M) if bench_close is not None else np.nan
b6 = pct_change(bench_close, WIN_6M) if bench_close is not None else np.nan

rows = []
sector_members = df_resolved.groupby("Sector")["Company"].apply(list).to_dict()
for sec, comps in sector_members.items():
    r1 = np.nanmean([_safe_ret(c, WIN_1M) for c in comps]) if comps else np.nan
    r3 = np.nanmean([_safe_ret(c, WIN_3M) for c in comps]) if comps else np.nan
    r6 = np.nanmean([_safe_ret(c, WIN_6M) for c in comps]) if comps else np.nan
    rs1 = r1 - b1 if not np.isnan(r1) and not np.isnan(b1) else np.nan
    rs3 = r3 - b3 if not np.isnan(r3) and not np.isnan(b3) else np.nan
    rs6 = r6 - b6 if not np.isnan(r6) and not np.isnan(b6) else np.nan
    sector_rs = 0.25*rs1 + 0.35*rs3 + 0.40*rs6
    rows.append({"Sector": sec, "Members": len(comps), "RS_1M": rs1, "RS_3M": rs3, "RS_6M": rs6, "SectorRS": sector_rs})

sector_df = pd.DataFrame(rows)

def _z(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    std = s.std(ddof=0)
    if std == 0 or np.isnan(std): return s*0
    return (s - s.mean()) / std

if not sector_df.empty:
    sector_df["SectorRS_z"] = _z(sector_df["SectorRS"])
    sector_df["SectorScore01"] = sector_df["SectorRS"].rank(pct=True)
else:
    sector_df["SectorRS_z"] = np.nan
    sector_df["SectorScore01"] = np.nan

sector_df.to_csv(SECTOR_RS_OUT, index=False)
sector_score_map = dict(zip(sector_df["Sector"], sector_df["SectorScore01"].fillna(0.5)))
sector_z_map     = dict(zip(sector_df["Sector"], sector_df["SectorRS_z"].fillna(0.0)))

print(f"💾 Sector RS saved: {SECTOR_RS_OUT}")
display(sector_df.sort_values('SectorRS', ascending=False))


💾 Sector RS saved: sector_rs_2025-09-28.csv


,Sector,Members,RS_1M,RS_3M,RS_6M,SectorRS,SectorRS_z,SectorScore01
2,Consumer Cyclical,28,-0.0005,0.0849,0.0957,0.0679,1.8633,1.0000
6,Financial Services,45,0.0261,-0.0068,0.0837,0.0376,0.8214,0.9167
5,Finance,1,-0.0683,-0.2241,0.3282,0.0358,0.7580,0.8333
8,Industrials,27,0.0320,-0.0137,0.0693,0.0309,0.5913,0.7500
0,Basic Materials,20,0.0153,0.0261,0.0203,0.0211,0.2519,0.6667
4,Energy,9,0.0384,-0.0046,0.0306,0.0202,0.2241,0.5833
11,Utilities,14,0.0423,0.0111,-0.0036,0.0130,-0.0245,0.5000
7,Healthcare,14,-0.0158,0.0312,0.0024,0.0079,-0.2001,0.4167
3,Consumer Defensive,13,-0.0233,0.0325,-0.0035,0.0042,-0.3297,0.3333
1,Communication Services,6,0.0262,-0.0362,0.0148,-0.0002,-0.4795,0.2500


In [7]:
# =========================================================================================
# CELL 6 — 10-pt momentum scores (DESC + Rank) with your Sector & Industry
# =========================================================================================
scores = []
for _, r in tech_df.iterrows():
    comp, tick = r.get("Company"), r.get("Ticker")
    if pd.isna(r.get("Close")):
        continue
    close, s20, s50, s200 = r.get("Close"), r.get("SMA20"), r.get("SMA50"), r.get("SMA200")
    v20, v50 = r.get("AvgVol20"), r.get("AvgVol50")
    h3m, h52 = r.get("High3M"), r.get("High52W")

    trend = price_trend_score(close, s20, s50, s200)
    rs    = relative_strength_score(closes_store.get(comp), bench_close) if bench_close is not None else 0.0
    volm  = volume_momentum_score(v20, v50)
    brk   = breakout_structure_score(close, h52, h3m)
    sec   = float(sector_score_map.get(r["Sector"], 0.5))

    total = (trend * (W_TREND/4.0) +
             rs    * (W_RS/2.0) +
             volm  * (W_VOL/1.5) +
             brk   * (W_BRK/1.5) +
             sec   * (W_SEC/1.0))

    scores.append({
        "RunDate": RUN_DATE,
        "Company": comp,
        "Ticker": tick,
        "Sector": r["Sector"],
        "Industry": r["Industry"],
        "Close": round(close, 2),
        "Trend(0-4)": round(trend, 2),
        "RS(0-2)": round(rs, 2),
        "Vol(0-1.5)": round(volm, 2),
        "Breakout(0-1.5)": round(brk, 2),
        "Sector(0-1)": round(sec, 2),
        "MomentumScore(0-10)": round(total, 2)
    })

scores_df = pd.DataFrame(scores)
scores_df = scores_df.sort_values(["MomentumScore(0-10)", "Company"], ascending=[False, True]).reset_index(drop=True)
scores_df.insert(0, "Rank", np.arange(1, len(scores_df) + 1))

display(scores_df.head(25))
scores_df.to_csv(SCORES_OUT, index=False)
print("💾 Saved (sorted):", SCORES_OUT)

# Append to historical log (for trend)
if os.path.exists(LOG_OUT):
    log_df = pd.read_csv(LOG_OUT)
    log_df = pd.concat([log_df, scores_df.drop(columns=["Rank"])], ignore_index=True)
else:
    log_df = scores_df.drop(columns=["Rank"]).copy()
log_df = log_df.drop_duplicates(subset=["RunDate", "Company"], keep="last").sort_values(["Company", "RunDate"])
log_df.to_csv(LOG_OUT, index=False)
trend = log_df.pivot_table(index="RunDate", columns="Company", values="MomentumScore(0-10)")
trend.to_csv(TREND_OUT)

print("📈 Log updated:", LOG_OUT, " | 📊 Trend saved:", TREND_OUT)
display(trend.tail(10))
print("✅ Cell 6 done")


,Rank,RunDate,Company,Ticker,Sector,Industry,Close,Trend(0-4),RS(0-2),Vol(0-1.5),Breakout(0-1.5),Sector(0-1),MomentumScore(0-10)
0,1,2025-09-28,ASHOKLEY,ASHOKLEY.NS,Industrials,Farm & Heavy Construction Machinery,141.9300,4.0000,2.0000,1.0000,1.0000,0.7500,8.7500
1,2,2025-09-28,EICHERMOT,EICHERMOT.NS,Consumer Cyclical,Auto Manufacturers,"7,047.0000",4.0000,2.0000,0.5000,1.0000,1.0000,8.5000
2,3,2025-09-28,MARUTI,MARUTI.NS,Consumer Cyclical,Auto Manufacturers,"16,286.0000",4.0000,2.0000,0.5000,1.0000,1.0000,8.5000
3,4,2025-09-28,ADANIGREEN,ADANIGREEN.NS,Utilities,Utilities - Renewable,"1,047.4000",4.0000,2.0000,1.5000,0.0000,0.5000,8.0000
4,5,2025-09-28,ADANIPOWER,ADANIPOWER.NS,Utilities,Utilities - Independent Power Producers,146.2500,4.0000,2.0000,1.5000,0.0000,0.5000,8.0000
5,6,2025-09-28,MOTHERSON,MOTHERSON.NS,Consumer Cyclical,Auto Parts,105.6600,4.0000,2.0000,1.0000,0.0000,1.0000,8.0000
6,7,2025-09-28,BAJFINANCE,BAJFINANCE.NS,Financial Services,Credit Services,985.1000,4.0000,2.0000,0.5000,0.5000,0.9200,7.9200
7,8,2025-09-28,INDIANB,INDIANB.NS,Financial Services,Banks - Regional,706.3500,4.0000,2.0000,0.0000,1.0000,0.9200,7.9200
8,9,2025-09-28,SBIN,SBIN.NS,Financial Services,Banks - Regional,856.9500,4.0000,2.0000,0.0000,1.0000,0.9200,7.9200
9,10,2025-09-28,JSWSTEEL,JSWSTEEL.NS,Basic Materials,Steel,"1,130.8000",4.0000,2.0000,0.0000,1.0000,0.6700,7.6700


💾 Saved (sorted): momentum_scores_2025-09-28.csv
📈 Log updated: momentum_log.csv  | 📊 Trend saved: momentum_trend.csv


Company,ABB,ABCAPITAL,ABFRL,ACC,ADANIENSOL,ADANIENT,ADANIGREEN,ADANIPORTS,ADANIPOWER,ALKEM,...,UNITDSPR,UPL,VBL,VEDL,VMM,VOLTAS,WAAREEENER,WIPRO,YESBANK,ZYDUSLIFE
RunDate,,,,,,,,,,,,,,,,,,,,,
2025-09-28,1.2500,4.9200,4.5000,1.6700,5.5000,6.0800,8.0000,6.7500,8.0000,6.4200,...,0.8300,2.1700,0.8300,6.6700,2.0000,2.5000,3.1700,0.1700,7.4200,3.4200


✅ Cell 6 done


In [8]:
# ============================================================
# CELL 7 — Add 12-1 momentum, vol, z-scores (cross-sectional)
# ============================================================
def ret_12_1(close: pd.Series) -> float:
    if close is None or len(close) < 252: return np.nan
    try:
        return float(close.iloc[-21] / close.iloc[-252] - 1.0)
    except Exception:
        return np.nan

def vol_12m(close: pd.Series) -> float:
    if close is None or len(close) < 252: return np.nan
    rets = close.pct_change().dropna().iloc[-252:]
    return float(rets.std() * math.sqrt(252))

raw_rows = []
for _, r in tech_df.iterrows():
    comp = r["Company"]; tick = r["Ticker"]; close = closes_store.get(comp)
    if close is None or close.empty: continue
    r12_1 = ret_12_1(close)
    vol12 = vol_12m(close)
    trend = price_trend_score(r["Close"], r["SMA20"], r["SMA50"], r["SMA200"])
    rs    = relative_strength_score(close, bench_close) if bench_close is not None else np.nan
    brk   = breakout_structure_score(r["Close"], r["High52W"], r["High3M"])
    volm  = volume_momentum_score(r["AvgVol20"], r["AvgVol50"])
    raw_rows.append({
        "Company": comp, "Ticker": tick, "Sector": r["Sector"], "Industry": r["Industry"],
        "Close": float(r["Close"]), "Ret12_1": r12_1, "Vol12m": vol12,
        "RiskAdjMom": (r12_1/vol12) if (vol12 and not np.isnan(vol12) and vol12>0) else np.nan,
        "Trend_raw": trend, "RS_raw": rs, "Breakout_raw": brk, "VolMom_raw": volm
    })

raw_df = pd.DataFrame(raw_rows)

def zscore(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    std = s.std(ddof=0)
    if std == 0 or np.isnan(std):
        return s * 0
    return (s - s.mean()) / std

z_df = raw_df.copy()
for col in ["Ret12_1","RiskAdjMom","Trend_raw","RS_raw","Breakout_raw","VolMom_raw"]:
    z_df[col+"_z"] = zscore(raw_df[col])

display(z_df.head())
z_df.to_csv(ZSCORES_OUT, index=False)
print("💾 Z-scores saved:", ZSCORES_OUT)
print("✅ Cell 7 done")


,Company,Ticker,Sector,Industry,Close,Ret12_1,Vol12m,RiskAdjMom,Trend_raw,RS_raw,Breakout_raw,VolMom_raw,Ret12_1_z,RiskAdjMom_z,Trend_raw_z,RS_raw_z,Breakout_raw_z,VolMom_raw_z
0,ABB,ABB.NS,Industrials,Specialty Industrial Machinery,"5,173.3999",-0.3888,0.3213,-1.2098,0.0000,0.5000,0.0000,0.0000,-1.4457,-1.5368,-1.1430,-0.4136,-0.3211,-0.7368
1,ABCAPITAL,ABCAPITAL.NS,Financial Services,Financial Conglomerates,278.9000,0.1913,0.3381,0.5657,3.0000,0.5000,0.5000,0.0000,1.4442,1.4904,0.8622,-0.4136,2.0573,-0.7368
2,ABFRL,ABFRL.NS,Consumer Cyclical,Apparel Manufacturing,84.9000,-0.7748,0.7507,-1.0320,1.0000,2.0000,0.0000,0.5000,-3.3689,-1.2337,-0.4746,1.7827,-0.3211,0.4612
3,ACC,ACC.NS,Basic Materials,Building Materials,"1,824.4000",-0.2663,0.2255,-1.1813,0.5000,0.5000,0.0000,0.0000,-0.8358,-1.4882,-0.8088,-0.4136,-0.3211,-0.7368
4,ADANIENSOL,ADANIENSOL.NS,Utilities,Utilities - Independent Power Producers,879.2000,-0.2413,0.5172,-0.4665,3.0000,1.5000,0.0000,0.5000,-0.7109,-0.2695,0.8622,1.0506,-0.3211,0.4612


💾 Z-scores saved: momentum_zscores_2025-09-28.csv
✅ Cell 7 done


In [9]:
# ===================================================
# CELL 8 — Sector z-scores (from Cell 5.5 artifacts)
# ===================================================
# We already computed sector_df, sector_z_map, sector_score_map in Cell 5.5
# Just display (and ensure availability for downstream)
display(sector_df.sort_values("SectorRS", ascending=False))
print("✅ Cell 8 done (sector RS/z from Cell 5.5)")


,Sector,Members,RS_1M,RS_3M,RS_6M,SectorRS,SectorRS_z,SectorScore01
2,Consumer Cyclical,28,-0.0005,0.0849,0.0957,0.0679,1.8633,1.0000
6,Financial Services,45,0.0261,-0.0068,0.0837,0.0376,0.8214,0.9167
5,Finance,1,-0.0683,-0.2241,0.3282,0.0358,0.7580,0.8333
8,Industrials,27,0.0320,-0.0137,0.0693,0.0309,0.5913,0.7500
0,Basic Materials,20,0.0153,0.0261,0.0203,0.0211,0.2519,0.6667
4,Energy,9,0.0384,-0.0046,0.0306,0.0202,0.2241,0.5833
11,Utilities,14,0.0423,0.0111,-0.0036,0.0130,-0.0245,0.5000
7,Healthcare,14,-0.0158,0.0312,0.0024,0.0079,-0.2001,0.4167
3,Consumer Defensive,13,-0.0233,0.0325,-0.0035,0.0042,-0.3297,0.3333
1,Communication Services,6,0.0262,-0.0362,0.0148,-0.0002,-0.4795,0.2500


✅ Cell 8 done (sector RS/z from Cell 5.5)


In [10]:
# ===========================================================================
# CELL 9 — Composite z-score rank (fund-style) & compare with 10-pt score
# ===========================================================================
# Use Sector_z from Cell 5.5
z_df["Sector_z"] = z_df["Sector"].map(lambda s: sector_z_map.get(s, 0.0))

merged = z_df.merge(scores_df[["Company","MomentumScore(0-10)"]], on="Company", how="left")
merged["Z_Total"] = (
    WZ_12_1   * merged["Ret12_1_z"].fillna(0) +
    WZ_TREND  * merged["Trend_raw_z"].fillna(0) +
    WZ_RS     * merged["RS_raw_z"].fillna(0) +
    WZ_BRK    * merged["Breakout_raw_z"].fillna(0) +
    WZ_VOL    * merged["VolMom_raw_z"].fillna(0) +
    WZ_SECTOR * merged["Sector_z"].fillna(0)
)

merged = merged.sort_values("Z_Total", ascending=False).reset_index(drop=True)
merged.insert(0, "Z_Rank", np.arange(1, len(merged)+1))
display(merged.head(25))
merged.to_csv(f"momentum_composite_{RUN_DATE}.csv", index=False)
print("💾 Composite z-score table saved: momentum_composite_", RUN_DATE)
print("✅ Cell 9 done")


,Z_Rank,Company,Ticker,Sector,Industry,Close,Ret12_1,Vol12m,RiskAdjMom,Trend_raw,...,VolMom_raw,Ret12_1_z,RiskAdjMom_z,Trend_raw_z,RS_raw_z,Breakout_raw_z,VolMom_raw_z,Sector_z,MomentumScore(0-10),Z_Total
0,1,EICHERMOT,EICHERMOT.NS,Consumer Cyclical,Auto Manufacturers,"7,047.0000",0.2467,0.2496,0.9886,4.0000,...,0.5000,1.7205,2.2115,1.5306,1.7827,4.4358,0.4612,1.8633,8.5000,2.0868
1,2,PAYTM,PAYTM.NS,Technology,Software - Infrastructure,"1,125.2000",0.7558,0.4674,1.6170,2.0000,...,0.0000,4.2568,3.2830,0.1938,0.3185,-0.3211,-0.7368,-1.2940,3.1700,2.0166
2,3,INDIANB,INDIANB.NS,Financial Services,Banks - Regional,706.3500,0.2412,0.3138,0.7685,4.0000,...,0.0000,1.6929,1.8364,1.5306,1.7827,4.4358,-0.7368,0.8214,7.9200,1.9088
3,4,MARUTI,MARUTI.NS,Consumer Cyclical,Auto Manufacturers,"16,286.0000",0.1569,0.2241,0.6999,4.0000,...,0.5000,1.2729,1.7194,1.5306,1.7827,4.4358,0.4612,1.8633,8.5000,1.8629
4,5,MUTHOOTFIN,MUTHOOTFIN.NS,Financial Services,Credit Services,"3,020.3000",0.2914,0.3141,0.9277,4.0000,...,0.0000,1.9431,2.1077,1.5306,1.7827,2.0573,-0.7368,0.8214,7.4200,1.7961
5,6,ASHOKLEY,ASHOKLEY.NS,Industrials,Farm & Heavy Construction Machinery,141.9300,0.0655,0.3013,0.2174,4.0000,...,1.0000,0.8176,0.8966,1.5306,1.7827,4.4358,1.6592,0.5913,8.7500,1.5680
6,7,BAJFINANCE,BAJFINANCE.NS,Financial Services,Credit Services,985.1000,0.1514,0.2649,0.5717,4.0000,...,0.5000,1.2458,1.5007,1.5306,1.7827,2.0573,0.4612,0.8214,7.9200,1.5074
7,8,BSE,BSE.NS,Finance,Stock Broking,"2,043.2000",0.6386,0.5695,1.1212,0.0000,...,0.5000,3.6729,2.4377,-1.1430,-1.1457,-0.3211,0.4612,0.7580,1.3300,1.5028
8,9,LTF,LTF.NS,Financial Services,Credit Services,236.2900,0.1588,0.3179,0.4995,3.5000,...,0.0000,1.2825,1.3777,1.1964,1.7827,2.0573,-0.7368,0.8214,6.9200,1.3990
9,10,JSWSTEEL,JSWSTEEL.NS,Basic Materials,Steel,"1,130.8000",0.0447,0.2432,0.1839,4.0000,...,0.0000,0.7141,0.8395,1.5306,1.7827,4.4358,-0.7368,0.2519,7.6700,1.3625


💾 Composite z-score table saved: momentum_composite_ 2025-09-26
✅ Cell 9 done


In [10]:
# ======================================================================
# CELL 10 — Sector-capped, turnover-aware target weights + trade plan
# ======================================================================
# Current portfolio (weights)
nav_rows = []
latest_price_map = dict(zip(tech_df["Company"], tech_df["Close"]))
for _, r in df_resolved.iterrows():
    comp, qty = r["Company"], r["Quantity"]
    px = latest_price_map.get(comp, np.nan)
    nav_rows.append({"Company": comp, "Quantity": qty, "Close": px, "Value": (qty or 0) * (px or 0)})
nav_df = pd.DataFrame(nav_rows)
TOTAL_NAV = float(nav_df["Value"].sum())
if TOTAL_NAV <= 0:
    print("⚠️ TOTAL_NAV is zero; using 1.0 to avoid division by zero. Please ensure Quantity is filled.")
    TOTAL_NAV = 1.0

curr_w = (nav_df.set_index("Company")["Value"] / TOTAL_NAV).fillna(0)

# Base target weights from composite Z if available; else from 10-pt Momentum
work = merged[["Company","Sector","Z_Total"]].copy() if 'merged' in globals() else scores_df[["Company","Sector","MomentumScore(0-10)"]].copy()
if 'merged' in globals():
    metric = work["Z_Total"].fillna(0)
else:
    metric = work["MomentumScore(0-10)"].fillna(0)

if TOP_N is not None:
    work = work.iloc[:TOP_N].copy()

# Convert metric to positive weights (softmax-like)
k = 1.0  # concentration knob
w_raw = np.exp(k * metric)
work["BaseWeight"] = w_raw / w_raw.sum()

def apply_caps(df, single_cap, sector_cap):
    df = df.copy()
    df["Weight"] = df["BaseWeight"]
    for _ in range(3):  # a few passes
        # single name
        over = df["Weight"] > single_cap
        if over.any():
            excess = (df.loc[over, "Weight"] - single_cap).sum()
            df.loc[over, "Weight"] = single_cap
            under = ~over
            if under.any() and df.loc[under, "Weight"].sum() > 0:
                df.loc[under, "Weight"] += excess * (df.loc[under, "Weight"] / df.loc[under, "Weight"].sum())
        # sector cap
        sec_sum = df.groupby("Sector")["Weight"].transform("sum")
        for sec in df.loc[sec_sum > sector_cap, "Sector"].unique():
            mask = df["Sector"].eq(sec)
            scale = sector_cap / df.loc[mask, "Weight"].sum()
            df.loc[mask, "Weight"] *= scale
        df["Weight"] /= df["Weight"].sum()
    df["Weight"] /= df["Weight"].sum()
    return df

targets = apply_caps(work[["Company","Sector","BaseWeight"]], SINGLE_NAME_CAP, SECTOR_CAP)

# Bring all companies (zero for the rest)
all_companies = nav_df["Company"].unique().tolist()
tw_map = dict(zip(targets["Company"], targets["Weight"]))
target_w = pd.Series({c: tw_map.get(c, 0.0) for c in all_companies})

# Turnover control
def portfolio_turnover(w0: pd.Series, w1: pd.Series) -> float:
    return 0.5 * float(np.abs(w1.reindex(w0.index, fill_value=0) - w0).sum())

turn = portfolio_turnover(curr_w, target_w)
scale = 1.0
if TURNOVER_CAP and turn > TURNOVER_CAP:
    scale = TURNOVER_CAP / turn
adj_w = curr_w + scale * (target_w - curr_w)
adj_w = adj_w.clip(lower=0)
if adj_w.sum() == 0:
    adj_w = curr_w
adj_w = adj_w / adj_w.sum()

# Lookups
_sector_lookup   = df_resolved.set_index("Company")["Sector"].to_dict()
_industry_lookup = df_resolved.set_index("Company")["Industry"].to_dict()

# Convert to trade plan
plan_rows = []
for c in all_companies:
    px = latest_price_map.get(c, np.nan)
    curr_value = curr_w.get(c, 0.0) * TOTAL_NAV
    tgt_value  = adj_w.get(c, 0.0) * TOTAL_NAV
    trade_val  = tgt_value - curr_value
    curr_qty   = float(nav_df.loc[nav_df["Company"].eq(c), "Quantity"].fillna(0).sum())
    tgt_qty    = 0 if np.isnan(px) or px==0 else int(round(tgt_value / px))
    plan_rows.append({
        "Company": c, "Sector": _sector_lookup.get(c, "UNKNOWN"), "Industry": _industry_lookup.get(c, ""),
        "Price": px, "CurrentWeight": round(curr_w.get(c,0.0),4), "TargetWeight": round(adj_w.get(c,0.0),4),
        "CurrentQty": int(curr_qty), "TargetQty": int(tgt_qty), "TradeQty": int(tgt_qty - curr_qty),
        "TradeValue": round(trade_val, 2)
    })

targets_out = pd.DataFrame({
    "Company": adj_w.index, "TargetWeight": adj_w.values,
    "Sector": [ _sector_lookup.get(c,"UNKNOWN") for c in adj_w.index ],
    "Industry": [ _industry_lookup.get(c,"") for c in adj_w.index ],
}).sort_values("TargetWeight", ascending=False)

plan_df = pd.DataFrame(plan_rows).sort_values("TargetWeight", ascending=False)

targets_out.to_csv(TARGETS_OUT, index=False)
plan_df.to_csv(REBAL_OUT, index=False)

print(f"💾 Target weights saved: {TARGETS_OUT}")
print(f"💾 Rebalance plan saved: {REBAL_OUT}")
print(f"Turnover (one-way) applied: {portfolio_turnover(curr_w, adj_w):.2%} | raw target was {turn:.2%} (scale={scale:.2f})")
display(targets_out.head(20))
display(plan_df.head(20))
print("✅ Cell 10 done")


💾 Target weights saved: target_weights_2025-09-28.csv
💾 Rebalance plan saved: rebalance_plan_2025-09-28.csv
Turnover (one-way) applied: 25.00% | raw target was 78.73% (scale=0.32)


,Company,TargetWeight,Sector,Industry
144,MRF,0.1695,Consumer Cyclical,Auto Parts
168,PAGEIND,0.0451,Consumer Cyclical,Apparel Manufacturing
72,BOSCHLTD,0.0419,Consumer Cyclical,Auto Parts
12,MARUTI,0.0370,Consumer Cyclical,Auto Manufacturers
88,SHREECEM,0.0316,Basic Materials,Building Materials
127,ASHOKLEY,0.0294,Industrials,Farm & Heavy Construction Machinery
48,EICHERMOT,0.0269,Consumer Cyclical,Auto Manufacturers
89,DIXON,0.0196,Technology,Consumer Electronics
66,SOLARINDS,0.0151,Basic Materials,Specialty Chemicals
58,ADANIGREEN,0.0149,Utilities,Utilities - Renewable


,Company,Sector,Industry,Price,CurrentWeight,TargetWeight,CurrentQty,TargetQty,TradeQty,TradeValue
144,MRF,Consumer Cyclical,Auto Parts,"148,545.0000",0.2380,0.1695,1,1,0,"-42,751.8200"
168,PAGEIND,Consumer Cyclical,Apparel Manufacturing,"41,240.0000",0.0661,0.0451,1,1,0,"-13,087.9400"
72,BOSCHLTD,Consumer Cyclical,Auto Parts,"38,150.0000",0.0611,0.0419,1,1,0,"-11,980.1300"
12,MARUTI,Consumer Cyclical,Auto Manufacturers,"16,286.0000",0.0261,0.0370,1,1,0,"6,828.3300"
88,SHREECEM,Basic Materials,Building Materials,"28,910.0000",0.0463,0.0316,1,1,0,"-9,170.2300"
127,ASHOKLEY,Industrials,Farm & Heavy Construction Machinery,141.9300,0.0002,0.0294,1,129,128,"18,185.8900"
48,EICHERMOT,Consumer Cyclical,Auto Manufacturers,"7,047.0000",0.0113,0.0269,1,2,1,"9,761.9100"
89,DIXON,Technology,Consumer Electronics,"17,511.0000",0.0281,0.0196,1,1,0,"-5,251.8700"
66,SOLARINDS,Basic Materials,Specialty Chemicals,"13,766.0000",0.0221,0.0151,1,1,0,"-4,355.6500"
58,ADANIGREEN,Utilities,Utilities - Renewable,"1,047.4000",0.0017,0.0149,1,9,8,"8,279.1200"


✅ Cell 10 done


In [11]:
# =========================================================================================
# CELL 11 — Portfolio composite momentum (value-weighted) + time-series log
# =========================================================================================
import os

# Build current NAV & weights from Quantity × latest Close
latest_px = dict(zip(tech_df["Company"], tech_df["Close"]))

nav_rows = []
for _, r in df_resolved.iterrows():
    comp = r["Company"]
    qty  = float(r.get("Quantity", 0) or 0)
    px   = float(latest_px.get(comp, np.nan))
    val  = 0.0 if (np.isnan(px) or np.isnan(qty)) else qty * px
    nav_rows.append({"Company": comp, "Quantity": qty, "Close": px, "Value": val})

nav_df = pd.DataFrame(nav_rows).dropna(subset=["Company"])
TOTAL_NAV = float(nav_df["Value"].sum())
if TOTAL_NAV <= 0:
    raise RuntimeError("TOTAL_NAV is zero/negative. Check 'Quantity' and fetched 'Close' prices.")

w = (nav_df.set_index("Company")["Value"] / TOTAL_NAV)

# Join scores (10-pt) and compute value-weighted averages
sc_cols = ["Trend(0-4)", "RS(0-2)", "Vol(0-1.5)", "Breakout(0-1.5)", "Sector(0-1)", "MomentumScore(0-10)"]
scored = scores_df.set_index("Company")[sc_cols]

def wavg(series: pd.Series, weights: pd.Series) -> float:
    s = series.copy()
    ww = weights.reindex(s.index)
    mask = s.notna() & ww.notna() & (ww > 0)
    if not mask.any(): return np.nan
    ww = ww[mask] / ww[mask].sum()
    return float((s[mask] * ww).sum())

portfolio_metrics = {col: wavg(scored[col], w) for col in sc_cols}

# Optional: portfolio composite z-score (if Cell 9 ran)
port_z = np.nan
if 'merged' in globals() and "Z_Total" in merged.columns:
    z_map = dict(zip(merged["Company"], merged["Z_Total"]))
    port_z = wavg(pd.Series(z_map).reindex(w.index), w)

record = {
    "RunDate": RUN_DATE, "Holdings": int((w > 0).sum()), "NAV": round(TOTAL_NAV, 2),
    "w_Trend(0-4)": round(portfolio_metrics["Trend(0-4)"], 4),
    "w_RS(0-2)": round(portfolio_metrics["RS(0-2)"], 4),
    "w_Vol(0-1.5)": round(portfolio_metrics["Vol(0-1.5)"], 4),
    "w_Breakout(0-1.5)": round(portfolio_metrics["Breakout(0-1.5)"], 4),
    "w_Sector(0-1)": round(portfolio_metrics["Sector(0-1)"], 4),
    "PortfolioMomentum10": round(portfolio_metrics["MomentumScore(0-10)"], 4),
    "PortfolioZ": round(port_z, 4) if not np.isnan(port_z) else np.nan,
}

if os.path.exists(PORTF_LOG_OUT):
    plog = pd.read_csv(PORTF_LOG_OUT)
    plog = pd.concat([plog, pd.DataFrame([record])], ignore_index=True)
else:
    plog = pd.DataFrame([record])

plog = plog.drop_duplicates(subset=["RunDate"], keep="last").sort_values("RunDate")
plog.to_csv(PORTF_LOG_OUT, index=False)

print(f"💾 Portfolio momentum log updated: {PORTF_LOG_OUT}")
display(pd.DataFrame([record]))
display(plog.tail(10))


💾 Portfolio momentum log updated: portfolio_momentum_log.csv


,RunDate,Holdings,NAV,w_Trend(0-4),w_RS(0-2),w_Vol(0-1.5),w_Breakout(0-1.5),w_Sector(0-1),PortfolioMomentum10,PortfolioZ
0,2025-09-28,200,"624,131.2100",2.0891,0.8959,0.3862,0.1863,0.7617,4.3191,NaN


,RunDate,Holdings,NAV,w_Trend(0-4),w_RS(0-2),w_Vol(0-1.5),w_Breakout(0-1.5),w_Sector(0-1),PortfolioMomentum10,PortfolioZ
0,2025-09-28,200,"624,131.2100",2.0891,0.8959,0.3862,0.1863,0.7617,4.3191,NaN


In [13]:
# =========================================================================================
# CELL 12 — 6M backtest: "score ≥ THRESHOLD at t₀" (equal-weight) vs Nifty
# =========================================================================================
import numpy as np, pandas as pd

THRESHOLD = 6.0         # pick names with 10-pt score >= this at t0
LB_DAYS   = WIN_6M      # 6 months ≈ 126 trading days

# Reuse resilient fetch history for all names
def fetch_hist_df(sym):
    rsym, hist = get_history_resilient(sym)
    return rsym, hist  # hist has columns: open high low close volume ...

def last_le_idx(idx, target_ts):
    pos = idx.searchsorted(pd.Timestamp(target_ts), side='right') - 1
    return int(pos) if pos >= 0 else None

def pct_at(series, end_pos, win):
    if series is None or end_pos is None: return np.nan
    if end_pos - win < 0: return np.nan
    try:
        return float(series.iloc[end_pos] / series.iloc[end_pos - win] - 1.0)
    except Exception:
        return np.nan

def trend_at(close, end_pos):
    s = close.iloc[:end_pos+1]
    if s.empty: return 0.0
    c = float(s.iloc[-1])
    sma = lambda n: float(s.rolling(n).mean().iloc[-1]) if len(s) >= n else np.nan
    return price_trend_score(c, sma(20), sma(50), sma(200))

def volmom_at(vol, end_pos):
    v = vol.iloc[:end_pos+1]
    if len(v) < 20: return 0.0
    v20 = float(v.rolling(20).mean().iloc[-1]) if len(v)>=20 else np.nan
    v50 = float(v.rolling(50).mean().iloc[-1]) if len(v)>=50 else np.nan
    return volume_momentum_score(v20, v50)

def breakout_at(close, end_pos):
    s = close.iloc[:end_pos+1]
    if s.empty: return 0.0
    c = float(s.iloc[-1])
    h3 = float(s.rolling(WIN_3M).max().iloc[-1]) if len(s)>=WIN_3M else float(s.max())
    h52= float(s.rolling(252).max().iloc[-1]) if len(s)>=252 else float(s.max())
    return breakout_structure_score(c, h52, h3)

def rs_at(close, bench_close, end_pos, b_end_pos):
    s1 = pct_at(close, end_pos, WIN_1M)
    s3 = pct_at(close, end_pos, WIN_3M)
    b1 = pct_at(bench_close, b_end_pos, WIN_1M)
    b3 = pct_at(bench_close, b_end_pos, WIN_3M)
    score = 0.0
    if not np.isnan(s1) and not np.isnan(b1):
        d = s1 - b1; score += 1.0 if d >= 0.05 else (0.5 if d >= 0.0 else 0.0)
    if not np.isnan(s3) and not np.isnan(b3):
        d = s3 - b3; score += 1.0 if d >= 0.05 else (0.5 if d >= 0.0 else 0.0)
    return max(0.0, min(2.0, score))

# choose t0 (≈ 6 months ago on benchmark calendar)
if bench_close is None or len(bench_close) <= LB_DAYS:
    raise RuntimeError("Not enough benchmark history for 6M backtest.")
t0_pos = len(bench_close) - 1 - LB_DAYS
t0_date = bench_close.index[t0_pos]
print("t₀ (benchmark calendar):", t0_date.date())

# sector score at t0 from your portfolio
hist_store = {}
sector_members = df_resolved.groupby("Sector")["Company"].apply(list).to_dict()
for _, rr in df_resolved.iterrows():
    comp, sym = rr["Company"], rr["Ticker"]
    if comp in hist_store: continue
    rsym, h = fetch_hist_df(sym)
    hist_store[comp] = (rsym, h.rename(columns=str.lower) if not h.empty else pd.DataFrame())

def close_series(comp):
    dfh = hist_store.get(comp, (None, pd.DataFrame()))[1]
    return dfh["close"] if "close" in dfh.columns else pd.Series(dtype=float)

def volume_series(comp):
    dfh = hist_store.get(comp, (None, pd.DataFrame()))[1]
    return dfh["volume"] if "volume" in dfh.columns else pd.Series(dtype=float)

b_end_pos = t0_pos

rows_sec = []
for sec, comps in sector_members.items():
    r1s, r3s, r6s = [], [], []
    for c in comps:
        cs = close_series(c)
        if cs.empty:
            r1s.append(np.nan); r3s.append(np.nan); r6s.append(np.nan); continue
        epos = last_le_idx(cs.index, t0_date)
        r1s.append(pct_at(cs, epos, WIN_1M))
        r3s.append(pct_at(cs, epos, WIN_3M))
        r6s.append(pct_at(cs, epos, WIN_6M))
    r1 = np.nanmean(r1s) if len(r1s) else np.nan
    r3 = np.nanmean(r3s) if len(r3s) else np.nan
    r6 = np.nanmean(r6s) if len(r6s) else np.nan
    b1 = pct_at(bench_close, b_end_pos, WIN_1M); b3 = pct_at(bench_close, b_end_pos, WIN_3M); b6 = pct_at(bench_close, b_end_pos, WIN_6M)
    rs1 = r1 - b1 if not np.isnan(r1) and not np.isnan(b1) else np.nan
    rs3 = r3 - b3 if not np.isnan(r3) and not np.isnan(b3) else np.nan
    rs6 = r6 - b6 if not np.isnan(r6) and not np.isnan(b6) else np.nan
    sector_rs = 0.25*rs1 + 0.35*rs3 + 0.40*rs6
    rows_sec.append({"Sector": sec, "SectorRS": sector_rs})

sector_t0 = pd.DataFrame(rows_sec)
def z_or_zero(s):
    s = pd.to_numeric(s, errors="coerce"); std = s.std(ddof=0)
    return (s - s.mean())/std if std not in (0, np.nan) else s*0
if not sector_t0.empty:
    sector_t0["SectorScore01"] = sector_t0["SectorRS"].rank(pct=True)
else:
    sector_t0["SectorScore01"] = np.nan

sector_score_t0 = dict(zip(sector_t0["Sector"], sector_t0["SectorScore01"].fillna(0.5)))

# compute each stock’s 10-pt score at t0 and forward return
pick_rows = []
for _, rr in df_resolved.iterrows():
    comp, sym, sec = rr["Company"], rr["Ticker"], rr["Sector"]
    rsym, h = hist_store.get(comp, (None, pd.DataFrame()))
    if h.empty:
        continue
    c, v = h["close"], h["volume"]
    end_pos = last_le_idx(c.index, t0_date)
    if end_pos is None or end_pos < max(WIN_1M, 20):
        continue

    trend = trend_at(c, end_pos)
    rs    = rs_at(c, bench_close, end_pos, b_end_pos)
    volm  = volmom_at(v, end_pos)
    brk   = breakout_at(c, end_pos)
    sec_s = float(sector_score_t0.get(sec, 0.5))  # 0..1

    total = (trend * (W_TREND/4.0) +
             rs    * (W_RS/2.0) +
             volm  * (W_VOL/1.5) +
             brk   * (W_BRK/1.5) +
             sec_s * (W_SEC/1.0))

    close_t0 = float(c.iloc[end_pos]); close_now = float(c.iloc[-1])
    ret = (close_now / close_t0 - 1.0) if close_t0 > 0 else np.nan

    pick_rows.append({
        "Company": comp, "Ticker": rsym or sym, "Sector": sec,
        "Score_t0": round(float(total), 2),
        "Trend_t0": round(float(trend), 2),
        "RS_t0": round(float(rs), 2),
        "Vol_t0": round(float(volm), 2),
        "Brk_t0": round(float(brk), 2),
        "SectorScore_t0": round(float(sec_s), 2),
        "Close_t0": round(close_t0, 2), "Close_now": round(close_now, 2),
        "Ret_%": round(100.0 * ret, 2),
    })

bt_df = pd.DataFrame(pick_rows)
picks = bt_df[bt_df["Score_t0"] >= THRESHOLD].copy().sort_values(["Score_t0","Company"], ascending=[False, True])

# benchmark return t0 → now
b_ret = float(bench_close.iloc[-1] / bench_close.iloc[b_end_pos] - 1.0) if len(bench_close) > b_end_pos else np.nan

# equal-weight portfolio return of picks
if not picks.empty:
    ew_ret = float((picks["Ret_%"] / 100.0).mean())
    hit_rate = float((picks["Ret_%"] > 0).mean()) if len(picks)>0 else np.nan
else:
    ew_ret, hit_rate = np.nan, np.nan

summary = {
    "RunDate": RUN_DATE, "T0Date": str(pd.to_datetime(t0_date).date()),
    "Threshold": THRESHOLD, "Picks": int(len(picks)),
    "EW_Return_%": round(100.0 * ew_ret, 2) if not np.isnan(ew_ret) else np.nan,
    "Nifty_Return_%": round(100.0 * b_ret, 2) if not np.isnan(b_ret) else np.nan,
    "Outperformance_%": round(100.0 * (ew_ret - b_ret), 2) if (not np.isnan(ew_ret) and not np.isnan(b_ret)) else np.nan,
    "HitRate_%": round(100.0 * hit_rate, 1) if not np.isnan(hit_rate) else np.nan,
}

picks.to_csv(BT6M_PICKS_OUT, index=False)
print(f"💾 Picks saved: {BT6M_PICKS_OUT} (n={len(picks)})")

if os.path.exists(BT6M_LOG_OUT):
    log = pd.read_csv(BT6M_LOG_OUT)
    log = pd.concat([log, pd.DataFrame([summary])], ignore_index=True)
else:
    log = pd.DataFrame([summary])

log = log.drop_duplicates(subset=["RunDate","T0Date","Threshold"], keep="last").sort_values(["RunDate","T0Date"])
log.to_csv(BT6M_LOG_OUT, index=False)
print(f"💾 Backtest summary appended: {BT6M_LOG_OUT}")

print("\n=== Backtest Summary (6M, score ≥", THRESHOLD, ") ===")
display(pd.DataFrame([summary]))
if not picks.empty:
    display(picks.head(20))
else:
    print("No picks met the threshold at t₀; try lowering THRESHOLD or verify history coverage.")


t₀ (benchmark calendar): 2025-03-25
💾 Picks saved: bt6m_picks_2025-09-26.csv (n=23)
💾 Backtest summary appended: bt6m_log.csv

=== Backtest Summary (6M, score ≥ 6.0 ) ===


,RunDate,T0Date,Threshold,Picks,EW_Return_%,Nifty_Return_%,Outperformance_%,HitRate_%
0,2025-09-26,2025-03-25,6.0000,23,7.1500,4.1700,2.9800,73.9000


,Company,Ticker,Sector,Score_t0,Trend_t0,RS_t0,Vol_t0,Brk_t0,SectorScore_t0,Close_t0,Close_now,Ret_%
14,KOTAKBANK,KOTAKBANK.NS,Financial Services,8.2500,4.0000,2.0000,0.5000,1.0000,0.7500,"2,170.0000","1,994.8000",-8.0700
88,SHREECEM,SHREECEM.NS,Basic Materials,8.0000,4.0000,1.5000,0.5000,1.0000,1.0000,"29,544.0500","28,910.0000",-2.1500
38,INDIGO,INDIGO.NS,Industrials,7.9200,4.0000,2.0000,0.5000,1.0000,0.4200,"5,006.7000","5,561.0000",11.0700
30,JSWSTEEL,JSWSTEEL.NS,Basic Materials,7.5000,4.0000,1.5000,0.0000,1.0000,1.0000,"1,061.7000","1,130.8000",6.5100
20,BAJAJFINSV,BAJAJFINSV.NS,Financial Services,7.2500,4.0000,1.5000,0.5000,0.5000,0.7500,"1,945.0500","2,003.2000",2.9900
61,BAJAJHLDNG,BAJAJHLDNG.NS,Financial Services,7.2500,4.0000,1.0000,0.5000,1.0000,0.7500,"12,521.5000","12,698.0000",1.4100
8,BAJFINANCE,BAJFINANCE.NS,Financial Services,7.2500,4.0000,1.5000,0.0000,1.0000,0.7500,906.7200,985.1000,8.6400
4,ICICIBANK,ICICIBANK.NS,Financial Services,7.2500,4.0000,1.0000,0.5000,1.0000,0.7500,"1,343.7000","1,359.6000",1.1800
39,TATASTEEL,TATASTEEL.NS,Basic Materials,7.0000,3.0000,2.0000,1.0000,0.0000,1.0000,156.7300,167.4000,6.8100
156,UPL,UPL.NS,Basic Materials,7.0000,4.0000,1.0000,0.0000,1.0000,1.0000,653.0000,648.0000,-0.7700


In [14]:
print("Benchmark days:", len(bench_close.index))
mend = pd.Series(bench_close.index).groupby([bench_close.index.year, bench_close.index.month]).max().tolist()
print("Total month-ends in data:", len(mend))
print("Chosen start date:", start.date())
print("Rebalance month-ends after start:", len(rebal_dates))
print("First/Last month-end:", mend[0].date(), mend[-1].date())

Benchmark days: 4423
Total month-ends in data: 217


NameError: name 'start' is not defined

In [ ]:
# =========================================================================================
# CELL 13 — Monthly walk-forward backtest (Top-N by composite Z, 200DMA, caps & costs)
# =========================================================================================
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# ---------- Config ----------
WF_START_DATE   = None          # e.g., "2019-01-01"; None = auto from data
N_TOP           = 20            # number of names to hold each rebalance
K_SOFTMAX       = 1.0           # >1 concentrates more into higher Z
NAME_CAP        = SINGLE_NAME_CAP
SECTOR_CAP_     = SECTOR_CAP
TURNOVER_CAP_   = TURNOVER_CAP  # one-way cap; if 0/None → no cap
COST_BPS_SIDE   = 15            # trading cost per side (bps). 15 = 0.15% per side
USE_EQUAL_WEIGHT= False         # if True: equal weight; else softmax(Z)

# ---------- Safety ----------
need = ['df_resolved','bench_close','get_history_resilient',
        'price_trend_score','breakout_structure_score','volume_momentum_score',
        'pct_change','WIN_1M','WIN_3M','WIN_6M']
missing = [x for x in need if x not in globals()]
assert not missing, f"Run earlier cells first. Missing: {missing}"

# ---------- Helpers ----------
def last_le_idx(idx, target_ts):
    pos = idx.searchsorted(pd.Timestamp(target_ts), side='right') - 1
    return int(pos) if pos >= 0 else None

def pct_at(series, end_pos, win):
    if series is None or end_pos is None: return np.nan
    if end_pos - win < 0: return np.nan
    try:
        return float(series.iloc[end_pos] / series.iloc[end_pos - win] - 1.0)
    except Exception:
        return np.nan

def trend_at(close, end_pos):
    s = close.iloc[:end_pos+1]
    if s.empty: return np.nan
    c = float(s.iloc[-1])
    sma = lambda n: float(s.rolling(n).mean().iloc[-1]) if len(s) >= n else np.nan
    return price_trend_score(c, sma(20), sma(50), sma(200))

def volmom_at(vol, end_pos):
    v = vol.iloc[:end_pos+1]
    if len(v) < 20: return np.nan
    v20 = float(v.rolling(20).mean().iloc[-1]) if len(v)>=20 else np.nan
    v50 = float(v.rolling(50).mean().iloc[-1]) if len(v)>=50 else np.nan
    return volume_momentum_score(v20, v50)

def breakout_at(close, end_pos):
    s = close.iloc[:end_pos+1]
    if s.empty: return np.nan
    c = float(s.iloc[-1])
    h3 = float(s.rolling(WIN_3M).max().iloc[-1]) if len(s)>=WIN_3M else float(s.max())
    h52= float(s.rolling(252).max().iloc[-1]) if len(s)>=252 else float(s.max())
    return breakout_structure_score(c, h52, h3)

def rs_score_at(close, bench, end_pos, b_end_pos):
    s1 = pct_at(close, end_pos, WIN_1M); s3 = pct_at(close, end_pos, WIN_3M)
    b1 = pct_at(bench, b_end_pos, WIN_1M); b3 = pct_at(bench, b_end_pos, WIN_3M)
    score = 0.0
    if not np.isnan(s1) and not np.isnan(b1):
        d = s1 - b1; score += 1.0 if d >= 0.05 else (0.5 if d >= 0.0 else 0.0)
    if not np.isnan(s3) and not np.isnan(b3):
        d = s3 - b3; score += 1.0 if d >= 0.05 else (0.5 if d >= 0.0 else 0.0)
    return max(0.0, min(2.0, score))

def ret_12_1_at(close, end_pos):
    if end_pos is None or end_pos - 252 < 0: return np.nan
    try:
        return float(close.iloc[end_pos-21] / close.iloc[end_pos-252] - 1.0)
    except Exception:
        return np.nan

def zscore(col):
    s = pd.to_numeric(col, errors='coerce')
    std = s.std(ddof=0)
    if std == 0 or np.isnan(std): return s*0
    return (s - s.mean()) / std

def build_sector_groups(sector_map):
    df = pd.DataFrame({"Company": list(sector_map.keys()),
                       "Sector":  [sector_map[k] for k in sector_map.keys()]})
    return {sec: g["Company"].tolist() for sec, g in df.groupby("Sector")}

def sector_scores_at(date_t, hist_store, bench_close, sector_map):
    groups = build_sector_groups(sector_map)
    rows = []
    for sec, comps in groups.items():
        r1, r3, r6 = [], [], []
        for c in comps:
            dfh = hist_store.get(c, (None, pd.DataFrame()))[1]
            if dfh.empty or "close" not in dfh.columns:
                r1.append(np.nan); r3.append(np.nan); r6.append(np.nan); continue
            cs = dfh["close"]
            epos = last_le_idx(cs.index, date_t)
            r1.append(pct_at(cs, epos, WIN_1M))
            r3.append(pct_at(cs, epos, WIN_3M))
            r6.append(pct_at(cs, epos, WIN_6M))
        r1m = np.nanmean(r1) if len(r1) else np.nan
        r3m = np.nanmean(r3) if len(r3) else np.nan
        r6m = np.nanmean(r6) if len(r6) else np.nan
        bpos = last_le_idx(bench_close.index, date_t)
        b1 = pct_at(bench_close, bpos, WIN_1M); b3 = pct_at(bench_close, bpos, WIN_3M); b6 = pct_at(bench_close, bpos, WIN_6M)
        rs1 = r1m - b1 if not np.isnan(r1m) and not np.isnan(b1) else np.nan
        rs3 = r3m - b3 if not np.isnan(r3m) and not np.isnan(b3) else np.nan
        rs6 = r6m - b6 if not np.isnan(r6m) and not np.isnan(b6) else np.nan
        rows.append({"Sector": sec, "SectorRS": 0.25*rs1 + 0.35*rs3 + 0.40*rs6})
    df = pd.DataFrame(rows)
    if df.empty:
        return {}, {}
    df["SectorRS_z"] = zscore(df["SectorRS"])
    df["SectorScore01"] = df["SectorRS"].rank(pct=True)
    return dict(zip(df["Sector"], df["SectorRS_z"])), dict(zip(df["Sector"], df["SectorScore01"]))

def apply_caps(df, single_cap, sector_cap):
    df = df.copy()
    df["Weight"] = df["BaseWeight"]
    for _ in range(3):
        over = df["Weight"] > single_cap
        if over.any():
            excess = (df.loc[over,"Weight"] - single_cap).sum()
            df.loc[over,"Weight"] = single_cap
            under = ~over
            if under.any() and df.loc[under,"Weight"].sum() > 0:
                df.loc[under,"Weight"] += excess * (df.loc[under,"Weight"] / df.loc[under,"Weight"].sum())
        sec_sum = df.groupby("Sector")["Weight"].transform("sum")
        for sec in df.loc[sec_sum > sector_cap, "Sector"].unique():
            mask = df["Sector"].eq(sec)
            scale = sector_cap / df.loc[mask, "Weight"].sum()
            df.loc[mask, "Weight"] *= scale
        df["Weight"] /= df["Weight"].sum()
    df["Weight"] /= df["Weight"].sum()
    return df

def portfolio_turnover(w0, w1):
    delta = (w1.reindex(w0.index, fill_value=0) - w0).abs().sum()
    return float(delta), 0.5 * float(delta)

# Build history store for all names
hist_store = {}
for _, rr in df_resolved.iterrows():
    comp, sym = rr["Company"], rr["Ticker"]
    if comp in hist_store: continue
    rsym, h = get_history_resilient(sym)
    hist_store[comp] = (rsym, h.rename(columns=str.lower) if not h.empty else pd.DataFrame())

# Rebalance dates (month-end on benchmark calendar)
bidx = bench_close.index
if WF_START_DATE:
    start = pd.Timestamp(WF_START_DATE)
else:
    min_idx = min(len(bidx)-1, 252+210)  # leave >= 252+200d lookback
    start = bidx[max(0, min_idx)]
mend = pd.Series(bidx).groupby([bidx.year, bidx.month]).max().tolist()
rebal_dates = [d for d in mend if d >= start]
assert len(rebal_dates) >= 2, "Not enough month-ends to run backtest. Check history/WF_START_DATE."
print(f"Rebalance periods: {len(rebal_dates)-1} | from {rebal_dates[0].date()} to {rebal_dates[-1].date()}")

# Walk-forward
cost_rate = COST_BPS_SIDE / 10000.0
universe = df_resolved["Company"].unique().tolist()
sector_map = df_resolved.set_index("Company")["Sector"].to_dict()

w_curr = pd.Series(0.0, index=universe)
rows_perf, rows_hold = [], []

for i in range(len(rebal_dates)-1):
    t0, t1 = rebal_dates[i], rebal_dates[i+1]
    bpos = last_le_idx(bench_close.index, t0)

    sec_z_map, _sec_p_map = sector_scores_at(t0, hist_store, bench_close, sector_map)

    # signals at t0 (no look-ahead)
    sig_rows = []
    for comp in universe:
        dfh = hist_store.get(comp, (None, pd.DataFrame()))[1]
        if dfh.empty or "close" not in dfh.columns: continue
        c, v = dfh["close"], dfh.get("volume", pd.Series(dtype=float))
        epos = last_le_idx(c.index, t0)
        if epos is None or epos < 199:  # need 200dma
            continue
        sma200 = float(c.iloc[:epos+1].rolling(200).mean().iloc[-1])
        px     = float(c.iloc[epos])
        if np.isnan(sma200) or px <= sma200:
            continue

        ret12_1 = ret_12_1_at(c, epos)
        trend   = trend_at(c, epos)
        rs      = rs_score_at(c, bench_close, epos, bpos)
        brk     = breakout_at(c, epos)
        volm    = volmom_at(v, epos)
        sec_z   = float(sec_z_map.get(sector_map.get(comp, "UNKNOWN"), 0.0))

        sig_rows.append({
            "Company": comp, "Sector": sector_map.get(comp, "UNKNOWN"),
            "Ret12_1": ret12_1, "Trend_raw": trend, "RS_raw": rs,
            "Breakout_raw": brk, "VolMom_raw": volm, "Sector_z": sec_z,
        })

    sig = pd.DataFrame(sig_rows)
    if sig.empty:
        bret = float(bench_close.loc[t1] / bench_close.loc[t0] - 1.0)
        rows_perf.append({"Date": t1, "GrossRet": 0.0, "NetRet": 0.0, "BenchRet": bret,
                          "Turnover": 0.0, "Costs": 0.0, "NumSelected": 0})
        continue

    for col in ["Ret12_1","Trend_raw","RS_raw","Breakout_raw","VolMom_raw"]:
        sig[col+"_z"] = zscore(sig[col])

    sig["Z_Total"] = (WZ_12_1*sig["Ret12_1_z"].fillna(0) +
                      WZ_TREND*sig["Trend_raw_z"].fillna(0) +
                      WZ_RS*sig["RS_raw_z"].fillna(0) +
                      WZ_BRK*sig["Breakout_raw_z"].fillna(0) +
                      WZ_VOL*sig["VolMom_raw_z"].fillna(0) +
                      WZ_SECTOR*sig["Sector_z"].fillna(0))

    sig = sig.sort_values("Z_Total", ascending=False).reset_index(drop=True)
    picks = sig.head(min(N_TOP, len(sig))).copy()

    # base weights
    if USE_EQUAL_WEIGHT:
        picks["BaseWeight"] = 1.0 / len(picks)
    else:
        w_raw = np.exp(K_SOFTMAX * picks["Z_Total"].fillna(0))
        picks["BaseWeight"] = w_raw / w_raw.sum()

    # caps
    picks = apply_caps(picks[["Company","Sector","BaseWeight"]], NAME_CAP, SECTOR_CAP_)
    w_tgt = pd.Series(dict(zip(picks["Company"], picks["Weight"])), index=universe).fillna(0.0)

    # turnover & costs (with optional turnover cap)
    sum_abs_delta = (w_tgt - w_curr).abs().sum()
    one_way = 0.5 * float(sum_abs_delta)
    scale = min(1.0, (TURNOVER_CAP_ / one_way)) if (TURNOVER_CAP_ and one_way > TURNOVER_CAP_) else 1.0
    w_tgt_scaled = (w_curr + scale*(w_tgt - w_curr)).clip(lower=0)
    if w_tgt_scaled.sum() > 0: w_tgt_scaled /= w_tgt_scaled.sum()
    sum_abs_delta_scaled = (w_tgt_scaled - w_curr).abs().sum()
    cost = cost_rate * sum_abs_delta_scaled

    # period gross return t0→t1 using start weights w_tgt_scaled
    def stock_period_ret(comp):
        dfh = hist_store[comp][1]
        if dfh.empty or "close" not in dfh.columns: return np.nan
        c = dfh["close"]
        p0i = last_le_idx(c.index, t0); p1i = last_le_idx(c.index, t1)
        if p0i is None or p1i is None or p1i <= p0i: return np.nan
        return float(c.iloc[p1i] / c.iloc[p0i] - 1.0)

    rets = pd.Series({c: stock_period_ret(c) for c in universe})
    valid_mask = (~rets.isna()) & (w_tgt_scaled > 0)
    gross = float((w_tgt_scaled[valid_mask] * rets[valid_mask]).sum()) if valid_mask.any() else 0.0
    net = gross - cost

    # evolve weights to end-of-period
    if valid_mask.any():
        w_eop = w_tgt_scaled.copy()
        w_eop[valid_mask] = w_tgt_scaled[valid_mask] * (1.0 + rets[valid_mask])
        if w_eop.sum() > 0: w_eop /= w_eop.sum()
        w_curr = w_eop
    else:
        w_curr = w_tgt_scaled

    bret = float(bench_close.loc[t1] / bench_close.loc[t0] - 1.0)

    rows_perf.append({"Date": t1, "GrossRet": gross, "NetRet": net, "BenchRet": bret,
                      "Turnover": 0.5*sum_abs_delta_scaled, "Costs": cost,
                      "NumSelected": int((w_tgt_scaled>0).sum())})

    # snapshot holdings
    snap = w_tgt_scaled[w_tgt_scaled>0].sort_values(ascending=False)
    for comp, wt in snap.items():
        rows_hold.append({"Date": t0, "Company": comp, "Sector": sector_map.get(comp,"UNKNOWN"),
                          "Weight": float(wt)})

# Outputs
wf_periods = pd.DataFrame(rows_perf).sort_values("Date").reset_index(drop=True)
wf_periods["Equity"] = (1.0 + wf_periods["NetRet"]).cumprod()
wf_periods["BenchEquity"] = (1.0 + wf_periods["BenchRet"]).cumprod()

wf_holdings = pd.DataFrame(rows_hold)

wf_periods.to_csv("wf_periods.csv", index=False)
wf_holdings.to_csv("wf_holdings.csv", index=False)

print("💾 Saved: wf_periods.csv, wf_holdings.csv")
display(wf_periods.tail(10))


In [ ]:
# =============================================================================
# CELL 14 — Tear sheet (CAGR, Vol, Sharpe, MaxDD, IR, turnover, exposures)
# =============================================================================
import numpy as np, pandas as pd, matplotlib.pyplot as plt

assert 'wf_periods' in globals(), "Run Cell 13 first."

df = wf_periods.copy().sort_values("Date").reset_index(drop=True)

# Core metrics
n = len(df)
years = n / 12.0 if n>0 else np.nan
eq_end = float(df["Equity"].iloc[-1]) if n else np.nan
bench_end = float(df["BenchEquity"].iloc[-1]) if n else np.nan

ann_ret   = (eq_end ** (1/years) - 1.0) if (n and years>0 and eq_end>0) else np.nan
ann_vol   = float(df["NetRet"].std(ddof=0) * np.sqrt(12)) if n>1 else np.nan
sharpe    = (ann_ret / ann_vol) if (ann_vol and ann_vol>0) else np.nan

ann_bench = (bench_end ** (1/years) - 1.0) if (n and years>0 and bench_end>0) else np.nan
active    = df["NetRet"] - df["BenchRet"]
ir        = float((active.mean() * np.sqrt(12)) / (active.std(ddof=0) + 1e-12)) if n>1 else np.nan

# Max drawdown
roll_max = df["Equity"].cummax()
drawdown = df["Equity"]/roll_max - 1.0
max_dd   = float(drawdown.min())

# Turnover & costs
avg_tn   = float(df["Turnover"].mean()) if "Turnover" in df else np.nan
avg_cost = float(df["Costs"].mean())    if "Costs" in df else np.nan
hit_rate = float((df["NetRet"] > 0).mean()) if n else np.nan
avg_names= float(df["NumSelected"].mean()) if "NumSelected" in df else np.nan
best_m   = float(df["NetRet"].max()) if n else np.nan
worst_m  = float(df["NetRet"].min()) if n else np.nan

summary = pd.DataFrame([{
    "Periods": n, "Years": round(years,2),
    "CAGR%": round(100*ann_ret,2) if not np.isnan(ann_ret) else np.nan,
    "Vol%": round(100*ann_vol,2) if not np.isnan(ann_vol) else np.nan,
    "Sharpe(0%)": round(sharpe,2) if not np.isnan(sharpe) else np.nan,
    "BenchCAGR%": round(100*ann_bench,2) if not np.isnan(ann_bench) else np.nan,
    "InfoRatio": round(ir,2) if not np.isnan(ir) else np.nan,
    "MaxDD%": round(100*max_dd,2) if not np.isnan(max_dd) else np.nan,
    "HitRate%": round(100*hit_rate,1) if not np.isnan(hit_rate) else np.nan,
    "AvgNames": round(avg_names,1) if not np.isnan(avg_names) else np.nan,
    "AvgTurnover(one-way)%": round(100*avg_tn,1) if not np.isnan(avg_tn) else np.nan,
    "AvgCost%": round(100*avg_cost,2) if not np.isnan(avg_cost) else np.nan,
    "BestMonth%": round(100*best_m,2) if not np.isnan(best_m) else np.nan,
    "WorstMonth%": round(100*worst_m,2) if not np.isnan(worst_m) else np.nan,
}])

print("===== Walk-Forward Backtest — Tear Sheet =====")
display(summary)

# Last 24 months table
tail = df[["Date","NetRet","BenchRet","Equity","BenchEquity","NumSelected","Turnover","Costs"]].tail(24)
display(tail)

# Equity curve vs benchmark
plt.figure(figsize=(8,4.5))
plt.plot(df["Date"], df["Equity"], label="Portfolio")
plt.plot(df["Date"], df["BenchEquity"], label="Benchmark")
plt.title("Equity Curve (Net of Costs)")
plt.legend(); plt.grid(True); plt.tight_layout()
plt.show()

# Monthly active returns
plt.figure(figsize=(8,3.5))
plt.bar(df["Date"], active, width=20)
plt.title("Monthly Active Return (Portfolio − Benchmark)")
plt.axhline(0, linestyle="--")
plt.grid(True, axis='y'); plt.tight_layout()
plt.show()


In [ ]:
# === CELL 15 — Config & load universe (Top-10 weekly) ===
import pandas as pd, numpy as np, time, math, io, sys, datetime as dt
import yfinance as yf
from urllib.parse import urlencode

# --- Input Google Sheet: ID + GID (you gave this) ---
SHEET_ID = "1AEIpCI5prMzJ-NEa1A69c5yPVmTEmgYo3mjnqtzakBo"
GID      = "995974729"  # investable universe tab
SHEET_CSV_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={GID}"

# --- Benchmark ---
BENCHMARK_TICKER = "^NSEI"   # NIFTY 50 (Yahoo)

# --- Backtest parameters ---
TOP_N             = 10
REBAL_FREQ        = "W-FRI"  # weekly, Friday
TCOST_BPS_SIDE    = 20       # 20 bps per side (0.20%)
HIST_PERIOD_STOCK = "1500d"  # ~6 years for indicators + 3y test; use "max" if you like
BENCHMARK_PERIOD  = "max"

# --- Download universe from Google Sheet ---
df_uni = pd.read_csv(SHEET_CSV_URL)
# Normalize column names
df_uni.columns = [c.strip() for c in df_uni.columns]
# Guess sensible names
col_map = {
    "YahooSymbol": None,
    "Company": None,
    "Sector": None,
    "Industry": None
}
for c in df_uni.columns:
    lc = c.lower()
    if "yahoo" in lc and "symbol" in lc: col_map["YahooSymbol"] = c
    if lc in ("company","name","security","symbol"): col_map["Company"] = c if not col_map["Company"] else col_map["Company"]
    if "sector" in lc: col_map["Sector"] = c
    if "industry" in lc: col_map["Industry"] = c

missing = [k for k,v in col_map.items() if v is None]
assert "YahooSymbol" not in missing, "Cannot find 'YahooSymbol' column in the sheet."
if col_map["Company"] is None: df_uni["Company"] = df_uni[col_map["YahooSymbol"]]
else: df_uni["Company"] = df_uni[col_map["Company"]]

# Keep relevant cols
keep = ["Company", col_map["YahooSymbol"]]
if col_map["Sector"]:   keep.append(col_map["Sector"])
if col_map["Industry"]: keep.append(col_map["Industry"])
df_uni = df_uni[keep].copy()
df_uni.rename(columns={col_map["YahooSymbol"]: "YahooSymbol",
                       (col_map["Sector"] or "Sector"): "Sector",
                       (col_map["Industry"] or "Industry"): "Industry"}, inplace=True)
df_uni["Sector"]   = df_uni.get("Sector",   pd.Series(dtype=str)).fillna("UNKNOWN")
df_uni["Industry"] = df_uni.get("Industry", pd.Series(dtype=str)).fillna("")

# Basic cleaning
df_uni = df_uni.dropna(subset=["YahooSymbol"]).drop_duplicates(subset=["YahooSymbol"]).reset_index(drop=True)
print(f"Universe loaded: {len(df_uni)} symbols")
display(df_uni.head(10))


In [ ]:
# === CELL 16 — History fetch (stocks + benchmark) ===

def get_history_resilient(sym: str, period: str = HIST_PERIOD_STOCK):
    """
    Try provided Yahoo symbol; if empty, try common NSE/BSE/base fallbacks.
    Returns (resolved_symbol, DataFrame with cols: open, high, low, close, volume).
    """
    if not sym or str(sym).lower() in ("nan","none"):
        return None, pd.DataFrame()
    base = sym.split(".")[0]
    suffix = sym.split(".")[1] if "." in sym else ""

    candidates = [sym]
    if suffix != "NS": candidates.append(base + ".NS")
    candidates.append(base)
    if suffix != "BO": candidates.append(base + ".BO")

    seen, ordered = set(), []
    for c in candidates:
        if c and c not in seen:
            ordered.append(c); seen.add(c)

    for c in ordered:
        try:
            dfh = yf.Ticker(c).history(period=period, interval="1d", auto_adjust=False)
            if not dfh.empty:
                return c, dfh.rename(columns=str.lower)
        except Exception:
            pass
        time.sleep(0.15)
    return None, pd.DataFrame()

# Benchmark
bench_df = yf.Ticker(BENCHMARK_TICKER).history(period=BENCHMARK_PERIOD, interval="1d", auto_adjust=False)
assert not bench_df.empty, "Benchmark history empty."
bench_df = bench_df.rename(columns=str.lower)
bench_close = bench_df["close"].copy()

# Per-stock history store
hist_store, bad = {}, []
for _, r in df_uni.iterrows():
    comp, sym = r["Company"], r["YahooSymbol"]
    rsym, d = get_history_resilient(sym)
    if d.empty or "close" not in d:
        bad.append((comp, sym))
        continue
    hist_store[comp] = (rsym, d)  # keep resolved symbol & full history

print(f"Fetched histories: {len(hist_store)} ok | {len(bad)} failed")
if bad:
    print("Failed tickers:")
    for x in bad[:20]:
        print("  ", x)


In [ ]:
# === CELL 17 — Weekly scoring + backtest (Top-10, equal-weight) ===

def last_le_idx(index, dt_):
    """position of last index <= dt_ (or None)"""
    pos = index.searchsorted(dt_, side="right") - 1
    return int(pos) if pos >= 0 else None

def pct_at(series, pos, win):
    if pos is None or pos - win < 0: return np.nan
    a, b = float(series.iloc[pos]), float(series.iloc[pos - win])
    return (a / b) - 1.0 if b > 0 else np.nan

def zscore(x):
    x = np.array(x, dtype=float)
    m = np.nanmean(x); s = np.nanstd(x)
    if not np.isfinite(s) or s == 0: return np.full_like(x, np.nan)
    return (x - m) / s

# Rebalance dates: weekly Fridays from 3 years ago (with lookback headroom)
end_date = bench_close.index[-1]
start_raw = end_date - pd.DateOffset(years=3)
# need at least 252 days lookback before first signal
bidx = bench_close.index
first_pos = bidx.searchsorted(start_raw, side="left")
start_pos = max(first_pos, 252 + 10)  # +10 safety
rebal_dates = bench_close.iloc[start_pos:].resample(REBAL_FREQ).last().index.tolist()
assert len(rebal_dates) >= 10, "Too few weekly dates; check benchmark history."

# Precache sector map
company_to_sector = {r["Company"]: r.get("Sector","UNKNOWN") for _, r in df_uni.iterrows()}

# Portfolio state
equity = 1.0
bench_equity = 1.0
rows_periods, rows_holdings = [], []
prev_weights = {}  # {company: weight}

TCOST_PER_SIDE = TCOST_BPS_SIDE / 1e4  # 20 bps -> 0.002

for i, d in enumerate(rebal_dates[:-1]):  # skip last (we need next date to compute return)
    d_next = rebal_dates[i+1]

    # --- compute scores at date d ---
    # Benchmark returns
    bpos = last_le_idx(bench_close.index, d)
    b1 = pct_at(bench_close, bpos, 21)
    b3 = pct_at(bench_close, bpos, 63)
    b6 = pct_at(bench_close, bpos, 126)

    comps, RS, prox52, volmom, sector_raw = [], [], [], [], []
    per_comp_vals = {}

    # First pass: per-stock components
    for comp, (rsym, dfh) in hist_store.items():
        cs = dfh["close"]; vs = dfh.get("volume", pd.Series(dtype=float))
        pos = last_le_idx(cs.index, d)
        if pos is None:
            continue

        r1 = pct_at(cs, pos, 21)
        r3 = pct_at(cs, pos, 63)
        r6 = pct_at(cs, pos, 126)
        # Must have enough history
        if all(np.isnan([r1, r3, r6])):
            continue

        rsh = (0 if np.isnan(r1) or np.isnan(b1) else r1 - b1) * 0.25 \
            + (0 if np.isnan(r3) or np.isnan(b3) else r3 - b3) * 0.35 \
            + (0 if np.isnan(r6) or np.isnan(b6) else r6 - b6) * 0.40

        # 52w proximity (closer to high is better)
        hi252 = float(cs.iloc[max(0, pos-252):pos+1].max())
        px    = float(cs.iloc[pos])
        prox  = (px/hi252 - 1.0) if hi252 > 0 else np.nan

        # volume momentum
        if len(vs) > 50 and pos >= 50:
            v20 = float(vs.iloc[pos-19:pos+1].mean())
            v50 = float(vs.iloc[pos-49:pos+1].mean())
            vm  = (v20/v50 - 1.0) if v50 > 0 else np.nan
        else:
            vm = np.nan

        comps.append(comp)
        RS.append(rsh); prox52.append(prox); volmom.append(vm)

        per_comp_vals[comp] = {"pos": pos, "px": px}

    if len(comps) < TOP_N:
        # If very small universe after filtering, skip this week
        continue

    # Sector RS (z-scored across sectors)
    # Build sector -> members available this date
    df_tmp = pd.DataFrame({"Company": comps, "Sector": [company_to_sector.get(c,"UNKNOWN") for c in comps],
                           "RS": RS})
    sector_scores = {}
    for sec, g in df_tmp.groupby("Sector"):
        rs_list = g["RS"].values
        if len(rs_list) == 0:
            continue
        sector_scores[sec] = float(np.nanmean(rs_list))

    if sector_scores:
        sec_df = pd.DataFrame({"Sector": list(sector_scores.keys()),
                               "SecRS":   list(sector_scores.values())})
        sec_df["SecZ"] = zscore(sec_df["SecRS"])
        sectorZ_map = dict(zip(sec_df["Sector"], sec_df["SecZ"]))
    else:
        sectorZ_map = {}

    # Z-scores for components
    z_RS    = zscore(RS)
    z_52    = zscore(prox52)
    z_vmom  = zscore(volmom)
    secZ    = np.array([sectorZ_map.get(company_to_sector.get(c,"UNKNOWN"), 0.0) for c in comps], dtype=float)

    # Composite score
    score = 0.50*z_RS + 0.20*z_52 + 0.15*z_vmom + 0.15*secZ

    df_score = pd.DataFrame({
        "Company": comps,
        "Score":   score,
        "RS_z":    z_RS,
        "Prox52_z":z_52,
        "VolMom_z":z_vmom,
        "SectorZ": secZ
    }).dropna(subset=["Score"]).sort_values("Score", ascending=False).reset_index(drop=True)

    picks = df_score.head(TOP_N).copy()
    picks["Weight"] = 1.0 / TOP_N

    # --- compute realized (close-to-close) return for d -> d_next ---
    gross = 0.0
    valid_w_sum = 0.0
    for _, rr in picks.iterrows():
        comp = rr["Company"]; w = rr["Weight"]
        cs = hist_store[comp][1]["close"]
        pos0 = last_le_idx(cs.index, d)
        pos1 = last_le_idx(cs.index, d_next)
        if pos0 is None or pos1 is None or pos1 <= pos0:
            continue
        ret = float(cs.iloc[pos1]) / float(cs.iloc[pos0]) - 1.0
        gross += w * ret
        valid_w_sum += w
    if valid_w_sum > 0:
        gross = gross / valid_w_sum

    # Turnover & costs
    new_weights = {c: 1.0/TOP_N for c in picks["Company"].tolist()}
    # L1 turnover = sum |new - old| / 2
    keys = set(prev_weights.keys()).union(new_weights.keys())
    l1 = sum(abs(new_weights.get(k,0.0) - prev_weights.get(k,0.0)) for k in keys) / 2.0
    costs = l1 * (2 * TCOST_PER_SIDE)  # both sides

    net = gross - costs

    equity *= (1.0 + net)

    # Benchmark move same interval
    bpos0 = last_le_idx(bench_close.index, d)
    bpos1 = last_le_idx(bench_close.index, d_next)
    bret  = float(bench_close.iloc[bpos1]) / float(bench_close.iloc[bpos0]) - 1.0 if (bpos0 is not None and bpos1 is not None) else 0.0
    bench_equity *= (1.0 + bret)

    rows_periods.append({
        "Date": d, "GrossRet": gross, "NetRet": net, "BenchRet": bret,
        "Turnover": l1, "Costs": costs, "NumSelected": len(picks),
        "Equity": equity, "BenchEquity": bench_equity
    })

    for _, rr in picks.iterrows():
        rows_holdings.append({
            "Date": d, "Company": rr["Company"], "Weight": rr["Weight"],
            "Score": rr["Score"], "RS_z": rr["RS_z"],
            "Prox52_z": rr["Prox52_z"], "VolMom_z": rr["VolMom_z"],
            "SectorZ": rr["SectorZ"]
        })

    prev_weights = new_weights

# Save logs
wk_periods = pd.DataFrame(rows_periods).reset_index(drop=True)
wk_holdings = pd.DataFrame(rows_holdings).reset_index(drop=True)
wk_periods.to_csv("weekly_periods.csv", index=False)
wk_holdings.to_csv("weekly_holdings.csv", index=False)
print("💾 Saved weekly_periods.csv & weekly_holdings.csv")
display(wk_periods.tail(10))


In [ ]:
# === CELL 18 — Summary & plot (Top-10 weekly) ===
import matplotlib.pyplot as plt

assert not wk_periods.empty, "No periods produced."

# Equity stats
def drawdown_curve(e):
    peak = e.cummax()
    dd = e/peak - 1.0
    return dd

eq = wk_periods.set_index("Date")[["Equity","BenchEquity"]]
dd = drawdown_curve(eq["Equity"])

tot_ret = eq["Equity"].iloc[-1] - 1.0
b_tot   = eq["BenchEquity"].iloc[-1] - 1.0

# Annualized from weekly
wk_rets = wk_periods["NetRet"].values
b_wk    = wk_periods["BenchRet"].values
def ann(weekly):
    m = np.nanmean(weekly)
    v = np.nanstd(weekly)
    cagr = (1+m)**52 - 1
    vol  = v*np.sqrt(52)
    return cagr, vol
cagr, vol = ann(wk_rets)
b_cagr, b_vol = ann(b_wk)
sharpe = (np.nanmean(wk_rets)/np.nanstd(wk_rets))*np.sqrt(52) if np.nanstd(wk_rets)>0 else np.nan
mdd = dd.min()

print(f"Top-10 Weekly (3y) — Net of costs")
print(f"Total Return: {tot_ret:.2%} | Benchmark: {b_tot:.2%}")
print(f"CAGR: {cagr:.2%} | Vol: {vol:.2%} | Sharpe (0% rf): {sharpe:.2f} | Max DD: {mdd:.2%}")

# Plot equity
plt.figure(figsize=(9,5))
plt.plot(eq.index, eq["Equity"], label="Portfolio")
plt.plot(eq.index, eq["BenchEquity"], label="Benchmark")
plt.title("Equity Curve (Net of Costs) — Top-10 Weekly")
plt.legend(); plt.grid(True); plt.show()

# Also save equity to CSV for your records
eq.reset_index().to_csv("weekly_equity.csv", index=False)
print("💾 Saved weekly_equity.csv")


In [ ]:
# ===== Cell 19 — Build weekly_trades.csv (now with PnL_%) =====
import pandas as pd
import numpy as np


# Ensure df_trades exists in this cell
if 'df_trades' not in globals():
    if os.path.exists("weekly_trades.csv"):
        df_trades = pd.read_csv("weekly_trades.csv", parse_dates=["BuyDate","SellDate"])
    elif 'trades' in globals():  # fallback if you still have the list of dicts in memory
        df_trades = pd.DataFrame(trades)
    else:
        raise RuntimeError("No df_trades/trades found. Run the backtest cell that builds the trade log first.")

# If your trades are already in df_trades, skip the constructor below.
# Otherwise, build df_trades from your list of trade dicts `trades`:
# trades should have keys: Company, BuyDate, SellDate, Quantity, BuyPrice, SellPrice
# df_trades = pd.DataFrame(trades)

# Ensure dtypes (safe to run even if already correct)
df_trades["BuyDate"]   = pd.to_datetime(df_trades["BuyDate"])
df_trades["SellDate"]  = pd.to_datetime(df_trades["SellDate"])
for c in ["Quantity","BuyPrice","SellPrice"]:
    df_trades[c] = pd.to_numeric(df_trades[c], errors="coerce")

# Derived fields
df_trades["HoldingDays"] = (df_trades["SellDate"] - df_trades["BuyDate"]).dt.days
df_trades["BuyValue"]    = df_trades["BuyPrice"] * df_trades["Quantity"]
df_trades["SellValue"]   = df_trades["SellPrice"] * df_trades["Quantity"]
df_trades["PnL"]         = df_trades["SellValue"] - df_trades["BuyValue"]

# ✅ NEW: PnL% (gross) — add this line right after PnL
df_trades["PnL_%"] = (df_trades["PnL"] / df_trades["BuyValue"] * 100).where(df_trades["BuyValue"] > 0)

# (Optional) tidy columns
cols = ["Company","BuyDate","SellDate","HoldingDays","Quantity",
        "BuyPrice","SellPrice","BuyValue","SellValue","PnL","PnL_%"]
df_trades = df_trades[[c for c in cols if c in df_trades.columns]]

# Save
df_trades.to_csv("weekly_trades.csv", index=False)
print(f"💾 Saved weekly_trades.csv with {len(df_trades)} rows")



In [ ]:
# === CELL 20 — Compare weekly trades vs benchmark (Nifty 50) [TZ-FIXED] ===
import pandas as pd, numpy as np, math, matplotlib.pyplot as plt
import yfinance as yf

# ---- Config ----
BENCH_SYMBOL = "^NSEI"   # or "NIFTYBEES.NS"
TRADES_CSV   = "weekly_trades.csv"
OUT_CSV      = "weekly_vs_bench.csv"

# ---- Guards & inputs ----
assert 'hist_store' in globals(), "hist_store (price history by Company) not found. Run the earlier data cells first."
assert 'last_le_idx' in globals(), "last_le_idx helper missing. Run earlier cells where it was defined."
trades_df = pd.read_csv(TRADES_CSV, parse_dates=["BuyDate","SellDate"])
assert not trades_df.empty, "weekly_trades.csv is empty."

# --- Helper: make any DatetimeIndex tz-naive safely
def _naive_index(s: pd.Series) -> pd.Series:
    idx = s.index
    if hasattr(idx, "tz") and idx.tz is not None:
        s = s.copy()
        s.index = idx.tz_localize(None)
    return s

# Make trade dates tz-naive
trades_df["BuyDate"]  = pd.to_datetime(trades_df["BuyDate"]).dt.tz_localize(None)
trades_df["SellDate"] = pd.to_datetime(trades_df["SellDate"]).dt.tz_localize(None)

# Weekly date grid (Fridays) spanning the trade horizon (tz-naive)
start = trades_df["BuyDate"].min().normalize()
end   = trades_df["SellDate"].max().normalize()
dates = pd.date_range(start, end, freq="W-FRI")  # tz-naive by default

# Helper to get close on/<= date from hist_store (force tz-naive index)
def px_at_close(company: str, date: pd.Timestamp):
    cs = hist_store[company][1]["close"]
    cs = _naive_index(cs)
    pos = last_le_idx(cs.index, pd.Timestamp(date))  # both tz-naive now
    if pos is None:
        return np.nan
    return float(cs.iloc[pos])

# Portfolio weekly valuation: sum(qty * price) for all trades open on each date
def portfolio_value_on(d):
    # positions open if BuyDate <= d <= SellDate (we exit at close on SellDate)
    mask = (trades_df["BuyDate"] <= d) & (trades_df["SellDate"] >= d)
    if not mask.any():
        return np.nan
    open_tr = trades_df.loc[mask, ["Company","Quantity"]]
    vals = []
    for _, row in open_tr.iterrows():
        px = px_at_close(row["Company"], d)
        if np.isfinite(px):
            vals.append(px * row["Quantity"])
    return float(np.sum(vals)) if len(vals) else np.nan

port_vals = pd.Series({d: portfolio_value_on(d) for d in dates}).sort_index()
first_idx = port_vals.first_valid_index()
assert first_idx is not None, "No valid portfolio valuations — check trades and price history."
port_equity = (port_vals / port_vals.loc[first_idx]).ffill()

# Benchmark: fetch and force tz-naive index
bench = yf.Ticker(BENCH_SYMBOL).history(period="10y", interval="1d", auto_adjust=False)["Close"].dropna()
bench = _naive_index(bench)

def bench_close_on(d):
    pos = last_le_idx(bench.index, pd.Timestamp(d))  # tz-naive compare
    if pos is None:
        return np.nan
    return float(bench.iloc[pos])

bench_vals = pd.Series({d: bench_close_on(d) for d in dates}).sort_index()
first_b = bench_vals.first_valid_index()
bench_equity = (bench_vals / bench_vals.loc[first_b]).ffill()

# Trim to the common span where both exist and portfolio has started
common = port_equity.index.union(bench_equity.index)
df = pd.DataFrame({
    "PortEquity": port_equity.reindex(common).ffill(),
    "BenchEquity": bench_equity.reindex(common).ffill(),
}).dropna()
df = df[df.index >= first_idx]  # start from portfolio inception

# Weekly returns and active
df["PortRet"]  = df["PortEquity"].pct_change()
df["BenchRet"] = df["BenchEquity"].pct_change()
df["ActiveRet"] = df["PortRet"] - df["BenchRet"]

# Summary stats
yrs = (df.index[-1] - df.index[0]).days / 365.25
cagr_port  = (df["PortEquity"].iloc[-1] / df["PortEquity"].iloc[0])**(1/yrs) - 1
cagr_bench = (df["BenchEquity"].iloc[-1] / df["BenchEquity"].iloc[0])**(1/yrs) - 1

wk = df["PortRet"].dropna()
wk_b = df["BenchRet"].dropna()
ann_vol_port  = wk.std() * np.sqrt(52)
ann_vol_bench = wk_b.std() * np.sqrt(52)

def max_dd(series):
    roll_max = series.cummax()
    dd = series/roll_max - 1.0
    return dd.min()

mdd_port  = max_dd(df["PortEquity"])
mdd_bench = max_dd(df["BenchEquity"])

hit_weeks = (df["PortRet"] > df["BenchRet"]).sum()
hit_rate  = hit_weeks / (len(df) - 1) * 100 if len(df) > 1 else np.nan

# Save CSV
out = df.copy()
out.index.name = "Date"
out.reset_index().to_csv(OUT_CSV, index=False)
print(f"💾 Saved {OUT_CSV} with {len(out)} weekly rows")

# correction
sharpe_port  = (wk.mean() / wk.std() * np.sqrt(52)) if wk.std()  > 0 else np.nan
sharpe_bench = (wk_b.mean()/ wk_b.std()* np.sqrt(52)) if wk_b.std()> 0 else np.nan


# Print summary
print("\n--- Summary (since portfolio inception in this 3y window) ---")
print(f"Period: {df.index[0].date()} → {df.index[-1].date()}  (~{yrs:.2f} years)")
print(f"Total Return  | Portfolio: {df['PortEquity'].iloc[-1]/df['PortEquity'].iloc[0]-1: .2%}   "
      f"Benchmark: {df['BenchEquity'].iloc[-1]/df['BenchEquity'].iloc[0]-1: .2%}")
print(f"CAGR          | Portfolio: {cagr_port: .2%}   Benchmark: {cagr_bench: .2%}")
print(f"Ann.Vol       | Portfolio: {ann_vol_port: .2%}   Benchmark: {ann_vol_bench: .2%}")
print(f"Max Drawdown  | Portfolio: {mdd_port: .2%}   Benchmark: {mdd_bench: .2%}")
print(f"Sharpe (rf=0) | Portfolio: {sharpe_port if wk.std()>0 else np.nan: .2f}   "
      f"Benchmark: {sharpe_bench if wk_b.std()>0 else np.nan: .2f}")
print(f"Hit-rate vs Bench (weekly): {hit_rate: .1f}%")

# Plot equity curves
plt.figure(figsize=(9,4.8))
plt.plot(df.index, df["PortEquity"], label="Portfolio")
plt.plot(df.index, df["BenchEquity"], label="Benchmark")
plt.title("Weekly Equity Curve (Normalized)")
plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout()
plt.show()
